#### Imports

In [ ]:
from typing import Union
from typing_extensions import NotRequired, TypedDict
from functools import reduce

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr


from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    set_custom_rcParams,
    handler_map_alpha,
)

from sdm_eurec4a import RepositoryPath

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])

In [ ]:
def label_from_attrs(da: xr.DataArray, return_name=True, return_units=True):
    try:
        name = f"{da.attrs['long_name']}"
    except KeyError:
        name = f"{da.name}"

    if "units" in da.attrs:
        units = f"{da.attrs['units']}"
        if "$" not in units:
            units = f"${units}$"

        units = units.replace("$", " ")
        units = rf"$\left[ {units} \right]$"
    else:
        units = "[???]"

    if return_name == True and return_units == True:
        return f"{name} {units}"
    elif return_name == True and return_units == False:
        return f"{name}"
    elif return_name == False and return_units == True:
        return f"{units}"
    else:
        return ""

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

subdata_dir = "output_v3.5"
data_path = Path("/home/m/m301096/CLEO/data/") / subdata_dir

ds_subpath = "combined/eulerian_dataset_combined_v2.nc"

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / subdata_dir
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)


microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/thesis/results/output_v3.5


# Set time slice to use for temporal mean and median

In [ ]:
time_slice = slice(1500, 3590)  # seconds
radius_split = 45  # µm
radius_slice = slice(1e0, None)  # µm

# Load datasets

In [ ]:
class MicrophysicDict(TypedDict):
    dataset: xr.Dataset
    microphysics: str
    path: Path
    linestyle: Union[str, tuple]
    color: str


class OptionalDictOfMicrophysicDict(TypedDict):
    null_microphysics: NotRequired[MicrophysicDict]
    condensation: NotRequired[MicrophysicDict]
    collision_condensation: NotRequired[MicrophysicDict]
    coalbure_condensation_large: NotRequired[MicrophysicDict]
    coalbure_condensation_small: NotRequired[MicrophysicDict]


class DictOfMicrophysicDict(TypedDict):
    null_microphysics: MicrophysicDict
    condensation: MicrophysicDict
    collision_condensation: MicrophysicDict
    coalbure_condensation_large: MicrophysicDict
    coalbure_condensation_small: MicrophysicDict


data_dict = DictOfMicrophysicDict(
    null_microphysics=MicrophysicDict(
        microphysics="Null microphysics",
        linestyle=":",
        color="black",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    condensation=MicrophysicDict(
        microphysics="Condensation/Evaporation",
        linestyle="-",
        color="black",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    collision_condensation=MicrophysicDict(
        microphysics="Coll-coal, cond./evap.",
        linestyle="--",
        color="black",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    coalbure_condensation_large=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=125)\nand cond./evap.",
        linestyle=(0, (3, 1, 1, 1, 1, 1)),
        color="black",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    coalbure_condensation_small=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=5)\nand cond./evap.",
        linestyle=(0, (3, 1, 1, 1, 1, 1)),
        color="black",
        path=Path(),
        dataset=xr.Dataset(),
    ),
)

for mp in data_dict:
    data_dict[mp]["path"] = data_path / f"{mp}" / ds_subpath

for key in data_dict:
    ds = xr.open_dataset(data_dict[key]["path"], chunks={"cloud_id": 2})
    # ds = ds.sel(cloud_id = [18, 301])
    ds.attrs.update(microphysics=data_dict[key]["microphysics"])
    ds.attrs.update(microphysics_short=key)

    data_dict[key]["dataset"] = ds

# ---------------------------------------------------- #
# Reindex the datasets to have the same radius bins
# ---------------------------------------------------- #

combined_radius_bins = reduce(
    np.union1d, [data_dict[mp]["dataset"]["radius_bins"].values for mp in data_dict]
)
fill_value = np.nan
print("Number of radius bins:", len(combined_radius_bins))
print("Fill value:", fill_value)
for mp in data_dict:
    data_dict[mp]["dataset"] = data_dict[mp]["dataset"].reindex(
        radius_bins=combined_radius_bins, fill_value=fill_value
    )

Number of radius bins: 67
Fill value: nan


### All clouds which are simulated

In [ ]:
intersect_cloud_ids = reduce(
    np.intersect1d, [data_dict[key]["dataset"]["cloud_id"].data for key in data_dict]
)
for mp in data_dict:
    print(mp, len(data_dict[mp]["dataset"]["cloud_id"]))

print("\nIntersect:", len(intersect_cloud_ids))
print("cloud_ids:", intersect_cloud_ids)

null_microphysics 94
condensation 78
collision_condensation 83
coalbure_condensation_large 89
coalbure_condensation_small 91

Intersect: 63
cloud_ids: [  9  11  18  20  21  65  67  68  71  72  73  74  88  94 110 113 114 130
 142 194 197 198 199 201 203 205 207 208 211 212 213 215 217 218 219 220
 221 222 223 224 230 232 233 235 236 237 292 293 295 296 301 303 305 306
 307 308 309 311 312 314 359 361 362]


In [ ]:
clouds_dict = {
    "222": dict(
        cloud_id=222,
        color="r",
    ),
    "142": dict(
        cloud_id=142,
        color="b",
    ),
}
for key in data_dict:
    ds = data_dict[key]["dataset"]
    for cloud_id in clouds_dict:
        cloud_id = clouds_dict[cloud_id]["cloud_id"]
        is_in = cloud_id in ds["cloud_id"]
        print(f"{cloud_id}, {key}, {is_in}")

222, null_microphysics, True
142, null_microphysics, True
222, condensation, True
142, condensation, True
222, collision_condensation, True
142, collision_condensation, True
222, coalbure_condensation_large, True
142, coalbure_condensation_large, True
222, coalbure_condensation_small, True
142, coalbure_condensation_small, True


### Add further variables e.g. latent cooling

In [ ]:
# NOTE: For now, it needs to be divided by 2 s to get values per second Bue to a bug.
def add_variables(
    ds: xr.Dataset, latent_heat_of_condensation: float = 2.265e6, time_slice=time_slice  # J kg-1
):
    # fix attributes
    ds["mass_represented"].attrs["long_name"] = "Mass"

    ds["radius_bins"].attrs["long_name"] = "Radius"
    ds["radius_bins"].attrs["units"] = "$\\mu m$"

    ds["relative_humidity"].attrs["long_name"] = "Relative humidity"
    ds["relative_humidity"].attrs["units"] = "$\\%$"

    ds["gridbox_thickness"] = ds["gridbox_top"] - ds["gridbox_bottom"]
    ds["cloud_altitude"] = ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])

    # It seems that xi was stored as an integer. This is not wanted, because nan values will just be large integers.
    ds["xi"] = ds["xi"].astype(float)
    ds["xi"] = ds["xi"].where(ds["xi"] < 1e12)

    ds["xi"].attrs["units"] = "$\\#$"
    ds["xi"].attrs["long_name"] = "Multiplicity"

    ds["xi_per_volume"] = ds["xi"] / ds["gridbox_volume"]
    ds["xi_per_volume"].attrs["long_name"] = ds["xi"].attrs["long_name"]
    ds["xi_per_volume"].attrs["units"] = "$\\# m^{-3}$"

    ds["number_superdroplets_per_volume"] = 1000 * ds["number_superdroplets"] / ds["gridbox_volume"]
    ds["number_superdroplets_per_volume"].attrs["units"] = "$10^3m^{-3}$"
    ds["number_superdroplets_per_volume"].attrs["long_name"] = "Number of superdroplets per volume"

    ds["mass_represented_per_volume"] = 1e3 * ds["mass_represented"] / ds["gridbox_volume"]
    ds["mass_represented_per_volume"].attrs["units"] = "$g m^{-3}$"
    ds["mass_represented_per_volume"].attrs["long_name"] = "Mass"

    ds["evaporation_full"] = 1e3 * ds["massdelta_condensation"].where(ds["sub_cloud_layer_mask"])
    ds["evaporation_full"].attrs["units"] = "$g m^{-3} s^{-1}$"
    ds["evaporation_full"].attrs["long_name"] = "Evaporation rate"
    ds["evaporation_full"].attrs["description"] = "Evaporation rate for all timesteps"

    ds["evaporation"] = (
        ds["evaporation_full"].sel(time=time_slice).mean("time", keep_attrs=True, skipna=True)
    )
    ds["evaporation"].attrs["units"] = "$g m^{-3} s^{-1}$"
    ds["evaporation"].attrs["long_name"] = "Evaporation rate"

    ds["latent_heating"] = (
        1e-3 * ds["evaporation"] * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_heating"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating"].attrs["long_name"] = "Latent heating"

    ds["latent_heating_full"] = (
        1e-3 * ds["evaporation_full"] * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_heating_full"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_full"].attrs["long_name"] = "Latent heating"
    ds["latent_heating_full"].attrs["description"] = "Latent heating for all timesteps"

    ds["latent_heating_mean"] = ds["latent_heating"].mean("gridbox", keep_attrs=True)
    ds["latent_heating_sum"] = (ds["latent_heating"] * ds["gridbox_thickness"]).sum(
        "gridbox", keep_attrs=True
    )
    ds["latent_heating_sum"].attrs["units"] = "$W m^{-2}$"
    ds["latent_heating_sum"].attrs["long_name"] = "Column int. latent heating"

    ds["latent_heating_radius_bins"] = (
        ds["mass_difference_per_volume"].where(ds["sub_cloud_layer_mask"]) * latent_heat_of_condensation
    )
    ds["latent_heating_radius_bins"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_radius_bins"].attrs["long_name"] = "Latent heating"

    ds["latent_heating_radius_bins_time_median"] = (
        ds["latent_heating_radius_bins"]
        .sel(time=time_slice)
        .median("time", keep_attrs=True, skipna=True)
    )
    ds["latent_heating_radius_bins_time_median"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_radius_bins_time_median"].attrs["long_name"] = "Latent heating"
    ds["latent_heating_radius_bins_time_median"].attrs[
        "description"
    ] = "Median latent heating for all timesteps"


for mp in data_dict:
    ds = data_dict[mp]["dataset"]
    add_variables(ds)

# Analysis of the domain and metrics

## Identify outliers


It seems the outlier is cloud ``296``

In [ ]:
null_microphysics: xr.Dataset = data_dict["null_microphysics"]["dataset"]
condensation: xr.Dataset = data_dict["condensation"]["dataset"]
collision_condensation: xr.Dataset = data_dict["collision_condensation"]["dataset"]
coalbure_condensation_large: xr.Dataset = data_dict["coalbure_condensation_large"]["dataset"]
coalbure_condensation_small: xr.Dataset = data_dict["coalbure_condensation_small"]["dataset"]

# fig, axs = plt.subplots(ncols=5, figsize=(15, 5))
# null_microphysics["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[0])
# condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[1])
# collision_condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[2])
# coalbure_condensation_large["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[3])
# coalbure_condensation_small["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[4])

# remove outlier cloud_id 296
cloud_id_selection = intersect_cloud_ids[intersect_cloud_ids != 296]

null_microphysics = null_microphysics.sel(cloud_id=cloud_id_selection)
condensation = condensation.sel(cloud_id=cloud_id_selection)
collision_condensation = collision_condensation.sel(cloud_id=cloud_id_selection)
coalbure_condensation_large = coalbure_condensation_large.sel(cloud_id=cloud_id_selection)
coalbure_condensation_small = coalbure_condensation_small.sel(cloud_id=cloud_id_selection)

In [ ]:
# fig, axs = plt.subplots(ncols=5, figsize=(15, 5))
# null_microphysics["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[0])
# condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[1])
# collision_condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[2])
# coalbure_condensation_large["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[3])
# coalbure_condensation_small["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[4])

# Understanding the relation between the lagrangian method and the monitoring by CLEO

It can be seen, that the 

Solved, by reruning the eulerian scirpt: For some reason, the breakup schemes shows 0 values at some timesteps.

#### One cloud example 

In [ ]:
cloud_id = 222
microphysic = "condensation"
time_slice = slice(1500, 3590)

ds = data_dict[microphysic]["dataset"].sel(cloud_id=cloud_id).sel(time=time_slice)

direct = 1e3 * ds["massdelta_condensation"]
lagrange = (
    1e3 * ds["mass_difference_per_volume"].sum("radius_bins", skipna=True).compute()
)  # .where(ds["sub_cloud_layer_mask"]).compute()

units = "$g m^{-3}s^{-1}$"

direct.attrs["units"] = units
lagrange.attrs["units"] = units

lagrange = lagrange.where(lagrange != 0)
fig, axs = plt.subplots(ncols=2, figsize=(10, 5))

vmin = -5e-5
vmax = 0e-5

axs[0].set_xlim(vmin, vmax)
axs[1].set_xlim(vmin, vmax)
axs[1].set_ylim(vmin, vmax)

# plt.pcolormesh(ds["time"], ds["gridbox"], ds.T, shading = "auto")
axs[0].plot(
    lagrange.T,
    ds["gridbox_coord3"],
    linestyle="None",
    marker=".",
    color="grey",
    alpha=0.1,
    linewidth=0.1,
)

axs[0].plot(
    direct.mean("time"),
    ds["gridbox_coord3"],
    linestyle="-",
    marker="o",
    linewidth=3,
    alpha=0.5,
    color="blue",
    label="Monitor evap. rate",
)

axs[0].plot(
    lagrange.mean("time"),
    ds["gridbox_coord3"],
    linestyle="-",
    marker="+",
    color="tab:orange",
    alpha=0.5,
    linewidth=3,
    label="Mean lagrange method",
)
axs[0].plot(
    lagrange.median("time"),
    ds["gridbox_coord3"],
    linestyle="-",
    marker="x",
    color="red",
    alpha=0.5,
    linewidth=3,
    label="Median lagrange method",
)

axs[0].legend()
axs[0].set_xlabel(f"Mass difference [{units}]")
axs[0].set_ylabel("Gridbox")

axs[0].set_title("Veritcal profile of evaporation rate")

# axs[0].set_xlim(-0.4e-11, 0.4e-11)

axs[1].scatter(lagrange, direct, marker=".", color="grey", alpha=0.1, label="Individual")
axs[1].plot([vmin, vmax], [vmin, vmax], color="blue", linestyle="-", alpha=0.5)
axs[1].scatter(
    lagrange.median("time"),
    direct.mean("time"),
    marker="x",
    color="red",
    alpha=0.5,
    label="gridbox median",
)
axs[1].scatter(
    lagrange.mean("time"),
    direct.mean("time"),
    marker="+",
    color="orange",
    alpha=0.5,
    label="gridbox mean",
)

axs[1].set_xlabel(f"Lagrange method {units}")
axs[1].set_ylabel(f"Monitor evap. rate {units}")
axs[1].set_title(f"Monitor evap. rate vs. Lagrange method")
axs[1].legend()

fig.suptitle(f"Cloud ID: {cloud_id}, Microphysics: {microphysic}")
fig.tight_layout()

fig.savefig(fig_dir / f"evaporation_rate_cloud_id_{cloud_id}_microphysic_{microphysic}.png", dpi=400)
fig.savefig(fig_dir / f"evaporation_rate_cloud_id_{cloud_id}_microphysic_{microphysic}.svg")

In [ ]:
ds = condensation.sel(time=time_slice).sel(cloud_id=142)
ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))

LH_full = ds["latent_heating_radius_bins"]
mask = (LH_full > LH_full.quantile(0.1, "time")) & (LH_full < LH_full.quantile(0.9, "time"))
LH_full_mask = LH_full.where(mask)

LH = ds["latent_heating_full"]


fig, ax = plt.subplots(figsize=(10, 8))

ax.set_ylim(-150, 100)

ax.plot(
    1e3 * LH_full.sum("radius_bins").T, color=[0.75, 0.75, 0.75], marker=".", linestyle="None", alpha=0.2
)
ax.plot(1e3 * LH.mean("time"), linewidth=3, color="b", label="CLEO mean")
ax.plot(1e3 * LH.median("time"), linewidth=3, linestyle=":", color="r", label="CLEO median")
ax.plot(
    1e3 * LH_full.mean("time").sum("radius_bins"),
    color=default_colors[0],
    linewidth=3,
    linestyle="-",
    label="Lagrange mean sum",
)
ax.plot(
    1e3 * LH_full.median("time").sum("radius_bins"),
    color=default_colors[1],
    linewidth=3,
    linestyle="-",
    label="Lagrange median sum",
)
ax.plot(
    1e3 * LH_full.quantile(0.9, "time").sum("radius_bins"),
    color=default_colors[3],
    linewidth=3,
    linestyle="-",
)
ax.plot(
    1e3 * LH_full.quantile(0.1, "time").sum("radius_bins"),
    color=default_colors[3],
    linewidth=3,
    linestyle="-",
)
ax.plot(
    1e3 * LH_full_mask.mean("time").sum("radius_bins"),
    color=default_colors[0],
    linewidth=3,
    linestyle="--",
    label="Lagrange mean sum mask",
)
ax.plot(
    1e3 * LH_full_mask.median("time").sum("radius_bins"),
    color=default_colors[1],
    linewidth=3,
    linestyle="--",
    label="Lagrange median sum mask",
)
ax.plot(
    1e3 * LH_full.sum("radius_bins").median("time"),
    color="k",
    linewidth=2,
    linestyle=":",
    alpha=0.6,
    label="Lagrange sum median",
)
ax.plot(
    1e3 * LH_full.sum("radius_bins").mean("time"),
    color="k",
    linewidth=2,
    linestyle="--",
    alpha=0.6,
    label="Lagrange sum mean",
)
ax.legend(ncols=2, loc="upper center")
ax.set_xlabel("gridbox")
ax.set_ylabel("Latent heating [mW m$^{-3}$]")

microphysics = ds.attrs["microphysics"]

ax.set_title(
    f"""
Comparison of latent heating for lagrange and montior method
Different calculations for lagrange method are shown Cloud ID: {cloud_id}
Microphysics: {microphysics}
"""
)
fig.tight_layout()
fig.savefig(fig_dir / f"latent_heating_cloud_id_{cloud_id}.svg")

In [ ]:
datas = [condensation, collision_condensation, coalbure_condensation_large, coalbure_condensation_small]

fig, axs = plt.subplots(ncols=4, nrows=1, figsize=(20, 6), sharex=True, sharey=True)
colors = default_colors

vmin = -0.1
vmax = 0

for idx, data in enumerate(datas):
    ds: xr.Dataset = (
        data.sel(time=time_slice).sel(gridbox=slice(1, None)).where(ds["sub_cloud_layer_mask"])
    )
    direct = ds["latent_heating"]
    lagrange = (
        ds["latent_heating_radius_bins"].sum("radius_bins", skipna=True, keep_attrs=True).compute()
    )  # .where(ds["sub_cloud_layer_mask"]).compute()

    axs[idx].set_title(f"{data.attrs['microphysics']}")

    axs[idx].plot([vmin, vmax], [vmin, vmax], color="blue", linestyle="-", alpha=0.5)

    axs[idx].scatter(
        lagrange.mean("time"), direct, marker=".", color="orange", alpha=0.5, label="gridbox mean"
    )

    axs[idx].scatter(
        lagrange.median("time"), direct, marker=".", color="red", alpha=0.5, label="gridbox median"
    )

    axs[idx].set_xlabel(f"Lagrange method {label_from_attrs(lagrange)}")
    axs[idx].set_ylabel(f"Monitor method {label_from_attrs(direct)}")
    axs[idx].legend()

    axs[idx].set_xlim(vmin, vmax)
    axs[idx].set_ylim(vmin, vmax)

    axs[idx].plot([vmin, vmax], [vmin, vmax], color="k", linestyle="-", alpha=0.5)

fig.suptitle(f"Evaporation energy rate. Comaprison of Lagrange method and monitor method")
fig.tight_layout()
fig.savefig(fig_dir / f"evaporation_energy_rate_comparison.svg")

In [ ]:
datas = [
    null_microphysics,
    condensation,
    collision_condensation,
    coalbure_condensation_large,
    coalbure_condensation_small,
]
latent_heat_of_condensation: float = (2.265e6,)  # J kg-1

fig, axs = plt.subplots(ncols=len(datas), nrows=1, figsize=(20, 6), sharex=True, sharey=True)
colors = default_colors

vmin = -0.1
vmax = 0

for idx, data in enumerate(datas):
    ds: xr.Dataset = (
        data.sel(time=time_slice).sel(gridbox=slice(1, None)).where(ds["sub_cloud_layer_mask"])
    )
    direct = ds["latent_heating_full"].sel(time=time_slice)
    lagrange = (
        ds["latent_heating_radius_bins"]
        .sum("radius_bins")
        .where(ds["sub_cloud_layer_mask"])
        .sel(time=time_slice)
    )
    # lagrange = lagrange.where(lagrange != 0)

    axs[idx].hist(
        np.ravel(lagrange),
        bins=np.arange(-0.2, 0.2, 0.0025),
        alpha=0.5,
        color="red",
        label="Lagrange method",
        density=False,
    )
    axs[idx].hist(
        np.ravel(direct),
        bins=np.arange(-0.2, 0.2, 0.0025),
        alpha=0.5,
        color="blue",
        label="Monitor method",
        density=False,
    )

    axs[idx].set_title(data.attrs["microphysics"])
    axs[idx].set_xlabel(label_from_attrs(direct))
    axs[idx].set_ylabel("Probability density")

    axs[idx].set_yscale("log")
    axs[idx].legend()

fig.suptitle(f"Histogram of latent heating. Comaprison of Lagrange method and CLEO's monitor output")
fig.tight_layout()
fig.savefig(fig_dir / f"latent_heating_histogram_comparison.svg")

# Understand when to better use the median or the mean

The distributions can be very much not normal


In [ ]:
ds = collision_condensation.sel(time=time_slice).sel(cloud_id=142)
ds = ds.sel(gridbox=5)


variable = "latent_heating_radius_bins"
vmin, vmax = -0.1, 0.1
norm = mcolors.SymLogNorm(linthresh=1e-6, linscale=1e-6, vmin=vmin, vmax=vmax)
cmap = "RdBu_r"

# variable = "mass_difference_per_volume"
# vmin, vmax = -1e-8, 0
# norm = mcolors.SymLogNorm(linthresh=1e-10, linscale=1e-6, vmin= vmin, vmax= vmax)
# cmap = "inferno"

# variable = "xi_per_volume"
# vmin, vmax = 1e-2, 1e2
# norm = mcolors.SymLogNorm(linthresh=vmin, linscale=1e-6, vmin=0, vmax=vmax)
# cmap = "inferno_r"


data = ds[variable]

fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(16, 14))


axs_data = axs[:, 0]
axs_mask = axs[:, 1]


axs_mask[0].set_title("Zero values")
axs_mask[0].set_ylabel(label_from_attrs(data["time"]))
axs_mask[0].set_xlabel(label_from_attrs(data["radius_bins"]))
pcm = axs_mask[0].pcolormesh(data["radius_bins"], data["time"], data == 0, cmap="binary")
fig.colorbar(pcm, ax=axs_mask[0], label="Zero values")

axs_mask[1].set_title("Nan values")
axs_mask[1].set_ylabel(label_from_attrs(data["time"]))
axs_mask[1].set_xlabel(label_from_attrs(data["radius_bins"]))
pcm = axs_mask[1].pcolormesh(data["radius_bins"], data["time"], data.isnull(), cmap="binary")
fig.colorbar(pcm, ax=axs_mask[1], label="nan values")

axs_data[0].set_title("Data")
axs_data[0].set_ylabel(label_from_attrs(data["time"]))
axs_data[0].set_xlabel(label_from_attrs(data["radius_bins"]))
pcm = axs_data[0].pcolormesh(
    data["radius_bins"], data["time"], data, norm=norm, cmap=cmap, alpha=1  # norm = mcolors.LogNorm(),
)
fig.colorbar(pcm, ax=axs_data[0], label=label_from_attrs(data))

axs_data[1].set_title("Median")
axs_data[1].set_ylabel(label_from_attrs(data["time"]))
axs_data[1].set_xlabel(label_from_attrs(data["radius_bins"]))
axs_data[1].pcolormesh(
    data["radius_bins"],
    data["time"],
    data.median("time", skipna=True).expand_dims(time=data["time"]),
    norm=norm,  # norm = mcolors.LogNorm(),
    cmap=cmap,
    alpha=1,
)
fig.colorbar(pcm, ax=axs_data[1], label=label_from_attrs(data))

axs_data[2].set_title("Mean")
axs_data[2].set_ylabel(label_from_attrs(data["time"]))
axs_data[2].set_xlabel(label_from_attrs(data["radius_bins"]))
axs_data[2].pcolormesh(
    data["radius_bins"],
    data["time"],
    data.mean("time", skipna=True).expand_dims(time=data["time"]),
    norm=norm,  # norm = mcolors.LogNorm(),
    cmap=cmap,
    alpha=1,
)
fig.colorbar(pcm, ax=axs_data[2], label=label_from_attrs(data))


for ax in axs.flatten():
    ax.set_xscale("log")


fig.suptitle(
    f"{ds.attrs['microphysics']} | {label_from_attrs(data, return_units=False)} | Cloud: {ds['cloud_id'].item()} | Gridbox: {ds['gridbox'].item()}"
)
fig.tight_layout()
fig.savefig(
    fig_dir
    / f"{ds.attrs['microphysics_short']}_{variable}_{ds['gridbox'].item()}_{ds['cloud_id'].item()}.png",
    dpi=400,
)

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


In [ ]:
variables = [
    "xi_per_volume",
    "number_superdroplets_per_volume",
    "mass_represented_per_volume",
    "latent_heating_radius_bins",
]

ds = collision_condensation.sel(time=time_slice).sel(cloud_id=142).sel(gridbox=5)

fig, axs = plt.subplots(ncols=len(variables), nrows=2, figsize=(15, 10))

for i, variable in enumerate(variables):
    data = ds[variable]
    for j in range(2):
        axs[j, i].plot(
            data["radius_bins"],
            data.T,
            linestyle="None",
            marker=".",
            alpha=0.05,
            c="grey",
        )
        axs[j, i].plot(
            data["radius_bins"],
            data.mean("time"),
            linestyle="None",
            marker="x",
            c="orange",
            label="Mean",
        )
        axs[j, i].plot(
            data["radius_bins"],
            data.median("time", skipna=True),
            linestyle="None",
            c="red",
            marker="+",
            label="Median",
        )
        axs[j, i].set_ylabel(label_from_attrs(ds[variable]))
        axs[j, i].legend(loc="upper center")
    axs[0, i].set_title(label_from_attrs(ds[variable], return_units=False))

for ax in axs.flatten():
    ax.set_xscale("log")
    ax.set_xlabel(label_from_attrs(ds["radius_bins"]))
for ax in axs[0, :].flatten():
    ax.set_yscale("linear")
for ax in axs[1, :].flatten():
    ax.set_yscale("symlog", linthresh=1e-12)


fig.tight_layout()
fig.suptitle(
    f"Cloud ID: {ds['cloud_id'].item()} | Gridbox: {ds['gridbox'].item()} | Microphysics: {ds.attrs['microphysics']}"
)
fig.savefig(
    fig_dir
    / f"{ds.attrs['microphysics_short']}_{ds['cloud_id'].item()}_gridbox_{ds['gridbox'].item()}.png",
    dpi=400,
)

**NOTE**

So with the plots above we can set a guideline on when to use ``Mean`` and ``Median``

| variable | ``Mean`` | ``Median`` |
| -------- | ---- | ------ |
| $\xi$ | x | o |
| Mass | x | o |
| Number SDs | x | o |
| Mass difference | o | x |
| Latent Heating | o | x |

# Input Thermodynamic profiles and PSD

``NOTE``

We can state that the **median** of the lagrangian method gives reasonable results compared to tghe direct output from the evaporation monitor of CLEO.

### Thermodynamic profiles

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(
    condensation["relative_humidity"].T,
    condensation["gridbox_coord3"].T,
    alpha=0.6,
    # color = "k"
)
ax.set_title("Relative humidity profiles of all clouds")
ax.set_xlabel(label_from_attrs(condensation["relative_humidity"]))
ax.set_ylabel(label_from_attrs(condensation["gridbox_coord3"]))

fig.savefig(fig_dir / f"relative_humidity_profiles.svg")

### PSD, Mass and Number of SDs at different heights

In [ ]:
selected_gridboxes = (
    condensation["max_gridbox"].compute(),
    condensation["max_gridbox"].compute() - 1,
    10,
    5,
    0,
)

labels = (
    "Cloud layer",
    "Top Sub cloud layer",
    f"{condensation['gridbox_coord3'].sel(gridbox = 10).isel(cloud_id= 0).values} m",
    f"{condensation['gridbox_coord3'].sel(gridbox = 5).isel(cloud_id= 0).values} m",
    f"{condensation['gridbox_coord3'].sel(gridbox = 0).isel(cloud_id= 0).values} m",
)


microphysics = "coalbure_condensation_large"
ds: xr.Dataset = data_dict[microphysics]["dataset"].sel(time=time_slice)

selected_gridboxes = (
    ds["max_gridbox"].compute(),
    ds["max_gridbox"].compute() - 1,
    10,
    5,
    0,
)

needed_gridboxes = np.unique(np.hstack(selected_gridboxes))

labels = (
    "Cloud layer",
    "Top Sub cloud layer",
    f"{ds['gridbox_coord3'].sel(gridbox = 10).isel(cloud_id= 0).values} m",
    f"{ds['gridbox_coord3'].sel(gridbox = 5).isel(cloud_id= 0).values} m",
    f"{ds['gridbox_coord3'].sel(gridbox = 0).isel(cloud_id= 0).values} m",
)

In [ ]:
# calculate the median of the PSD and mass for the selected gridboxes
top_psd: xr.DataArray = (
    ds["xi_per_volume"].sel(gridbox=needed_gridboxes).mean("time", keep_attrs=True).compute()
)
top_mass: xr.DataArray = (
    ds["mass_represented_per_volume"]
    .sel(gridbox=needed_gridboxes)
    .mean("time", keep_attrs=True)
    .compute()
)
top_nbr: xr.DataArray = (
    ds["number_superdroplets_per_volume"]
    .sel(gridbox=needed_gridboxes)
    .mean("time", keep_attrs=True)
    .compute()
)
top_nbr: xr.DataArray = top_nbr * 8
top_nbr.attrs["units"]: xr.DataArray = "$\\#/(8000 m^{3})$"

In [ ]:
fig, axs = plt.subplots(
    nrows=len(selected_gridboxes), ncols=3, figsize=(16, 12), sharex=True, sharey=False
)

fig.suptitle(f"Microphysics: {ds.attrs['microphysics']}")

for ax in axs[:, 0]:
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(0.8e0, 2e3)
    ax.set_ylim(1e-3, 1e4)
    # ax.set_xlabel("Radius [µm]")
    ax.set_ylabel(f"Num. Con. [{top_psd.attrs['units']}]")

for ax in axs[:, 1]:
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(0.8e0, 2e3)
    ax.set_ylim(1e-10, 2e-1)
    # ax.set_xlabel("Radius [µm]")
    ax.set_ylabel(f"Mass [{top_mass.attrs['units']}]")

for ax in axs[:, 2]:
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_xlim(0.8e0, 2e3)
    ax.set_ylim(0, 15)
    ax.set_ylabel(f"Nbr. SDs [{top_nbr.attrs['units']}]")

for ax in axs[:, -1]:
    ax.set_xlabel("Radius [µm]")


for idx, selected_gridbox in enumerate(selected_gridboxes):
    axs[idx, 0].plot(
        ds["radius_bins"],
        top_psd.sel(gridbox=selected_gridbox).T,
        alpha=0.6,
    )
    axs[idx, 0].set_ylabel(f"{labels[idx]}\nNum. Con. [{top_psd.attrs['units']}]")

    axs[idx, 1].plot(
        ds["radius_bins"],
        top_mass.sel(gridbox=selected_gridbox).T,
        alpha=0.6,
    )
    axs[idx, 2].plot(
        ds["radius_bins"],
        top_nbr.sel(gridbox=selected_gridbox).T,
        alpha=0.6,
    )

fig.tight_layout()
fig.savefig(fig_dir / f"psd_mass_nbr_at_different_heights_{microphysics}.svg")

In [ ]:
selected_gridbox = condensation["max_gridbox"]


fig, axs = plt.subplots(nrows=3, figsize=(8, 8), sharex=True)

fig.suptitle(f"Microphysics: {condensation.attrs['microphysics']}, Cloud Layer")

top_psd = condensation["xi"].sel(gridbox=selected_gridbox).median("time").compute()
top_psd = top_psd.where(top_psd > 0)
top_mass = condensation["mass_represented"].sel(gridbox=selected_gridbox).median("time").compute()
top_mass = top_mass.where(top_psd > 0)
top_nbr = (
    (condensation["number_superdroplets"] / condensation["gridbox_volume"] * 20 * 20 * 20)
    .sel(gridbox=selected_gridbox)
    .median("time")
    .compute()
)
top_nbr = top_nbr.where(top_psd > 0)
# for cloud_id in condensation["cloud_id"]:
#     selection = condensation.sel(cloud_id = cloud_id)

axs[0].plot(
    condensation["radius_bins"],
    top_psd.T,
    alpha=0.6,
    # color = "k"
)
axs[0].set_xscale("log")
axs[0].set_yscale("log")
# axs[1].set_yscale("log")
axs[0].set_xlim(0.8e0, 2e3)
axs[0].set_ylabel("Number conc. [m$^{-3}$]")

axs[1].plot(
    condensation["radius_bins"],
    top_mass.T,
    alpha=0.6,
    # color = "k"
)
axs[1].set_xlabel("Radius [µm]")
axs[1].set_ylabel("Mass [$g m^{-3}$]")

axs[2].plot(
    condensation["radius_bins"],
    top_nbr.T,
    alpha=0.6,
    # color = "k"
)
axs[2].set_ylim(0, 20)

axs[2].set_xlabel("Radius [µm]")
axs[2].set_ylabel("Number of SDs [8000m$^{-3}$]")

Text(0, 0.5, 'Number of SDs [8000m$^{-3}$]')

In [ ]:
selected_gridbox = 0


fig, axs = plt.subplots(nrows=3, figsize=(8, 8), sharex=True)

fig.suptitle(
    f"Microphysics: {condensation.attrs['microphysics']}, {condensation['gridbox_coord3'].sel(gridbox = selected_gridbox).mean().values} m"
)

top_psd = condensation["xi"].sel(gridbox=selected_gridbox).median("time").compute()
top_psd = top_psd.where(top_psd > 0)
top_mass = condensation["mass_represented"].sel(gridbox=selected_gridbox).median("time").compute()
top_mass = top_mass.where(top_psd > 0)
top_nbr = (
    (condensation["number_superdroplets"] / condensation["gridbox_volume"] * 20 * 20 * 20)
    .sel(gridbox=selected_gridbox)
    .median("time")
    .compute()
)
top_nbr = top_nbr.where(top_psd > 0)
# for cloud_id in condensation["cloud_id"]:
#     selection = condensation.sel(cloud_id = cloud_id)

axs[0].plot(
    condensation["radius_bins"],
    top_psd.T,
    alpha=0.6,
    # color = "k"
)
axs[0].set_xscale("log")
axs[0].set_yscale("log")
# axs[1].set_yscale("log")
axs[0].set_xlim(0.8e0, 2e3)
axs[0].set_ylabel("Number conc. [m$^{-3}$]")

axs[1].plot(
    condensation["radius_bins"],
    top_mass.T,
    alpha=0.6,
    # color = "k"
)
axs[1].set_xlabel("Radius [µm]")
axs[1].set_ylabel("Mass [$g m^{-3}$]")

axs[2].plot(
    condensation["radius_bins"],
    top_nbr.T,
    alpha=0.6,
    # color = "k"
)
axs[2].set_ylim(0, 20)

axs[2].set_xlabel("Radius [µm]")
axs[2].set_ylabel("Number of SDs [8000m$^{-3}$]")

Text(0, 0.5, 'Number of SDs [8000m$^{-3}$]')

# Precipitation

Non smoothed precipitation shows, that individual SDs are responsible for the surface precipiation.

Smoothing the timeseries gives chance to get a more robust value of the mean rain rate.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

surface_precipitation = (
    condensation["precipitation"].sum(dim="radius_bins").chunk(dict(cloud_id=-1)).compute()
)

ax.fill_between(
    surface_precipitation["time"],
    surface_precipitation.quantile(0.10, dim="cloud_id"),
    surface_precipitation.quantile(0.90, dim="cloud_id"),
    color=[0.9, 0.9, 0.9],
    alpha=1,
    label="10% - 90%",
)
ax.fill_between(
    surface_precipitation["time"],
    surface_precipitation.quantile(0.25, dim="cloud_id"),
    surface_precipitation.quantile(0.75, dim="cloud_id"),
    color=[0.75, 0.75, 0.75],
    alpha=1,
    label="25% - 75%",
)


ax.plot(
    surface_precipitation["time"],
    surface_precipitation.median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label="median",
)


for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.plot(
        surface_precipitation["time"],
        surface_precipitation.sel(cloud_id=cloud_id),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_ylabel(r"Surface precipitation $mm/h$")
ax.set_xlabel(r"Time $s$")

ax.legend(loc="upper left")

fig.savefig(fig_dir / "surface_precipitation_unsmoothed.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

surface_precipitation = (
    condensation["precipitation"]
    .sum(dim="radius_bins")
    .chunk(dict(cloud_id=-1))
    .rolling(time=50, center=True)
    .mean()
    .compute()
)

ax.fill_between(
    surface_precipitation["time"],
    surface_precipitation.quantile(0.10, dim="cloud_id"),
    surface_precipitation.quantile(0.90, dim="cloud_id"),
    color=[0.9, 0.9, 0.9],
    alpha=1,
    label="10% - 90%",
)
ax.fill_between(
    surface_precipitation["time"],
    surface_precipitation.quantile(0.25, dim="cloud_id"),
    surface_precipitation.quantile(0.75, dim="cloud_id"),
    color=[0.75, 0.75, 0.75],
    alpha=1,
    label="25% - 75%",
)


ax.plot(
    surface_precipitation["time"],
    surface_precipitation.median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label="median",
)


for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.plot(
        surface_precipitation["time"],
        surface_precipitation.sel(cloud_id=cloud_id),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_ylabel(r"Surface precipitation $mm/h$")
ax.set_xlabel(r"Time $s$")

ax.legend(loc="upper left")

fig.savefig(fig_dir / "surface_precipitation_smoothed.svg")

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


#### Histogram of Precipiation and its relation to the mass in the surface gridbox

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

ax.hist(
    surface_precipitation.sel(time=time_slice).mean("time"),
    bins=20,
    color="k",
    alpha=0.5,
    density=True,
    label="Histogram",
)
ax.axvline(
    surface_precipitation.sel(time=time_slice).mean("time").median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label=f"Median",
)
for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.axvline(
        surface_precipitation.sel(cloud_id=cloud_id).sel(time=time_slice).mean("time"),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_xlabel(r"Mean surface precipitation $mm/h$")
ax.set_ylabel("Density")
ax.set_title("Histogram of mean surface precipitation")

fig.savefig(fig_dir / "histogram_surface_precipitation.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

ax.hist(
    24 * surface_precipitation.sel(time=time_slice).mean("time"),
    bins=20,
    color="k",
    alpha=0.5,
    density=True,
    label="Histogram",
)
ax.axvline(
    24 * surface_precipitation.sel(time=time_slice).mean("time").median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label=f"Median",
)
for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.axvline(
        24 * surface_precipitation.sel(cloud_id=cloud_id).sel(time=time_slice).mean("time"),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_xlabel(r"Mean surface precipitation $mm/d$")
ax.set_ylabel("Density")
ax.set_title("Histogram of mean surface precipitation")

fig.savefig(fig_dir / "histogram_surface_precipitation_day.svg")

### Context to rain literature

The rain rates are siginificantly lower than “rain intensities of 0.9 mm h−1” ([Vogel et al., 2021, p. 16625](zotero://select/library/items/TNDVW8WC)) ([pdf](zotero://open-pdf/library/items/FZ2HISRL?page=17&annotation=PYMP44CU))

This might be due to the fact, that we did not sample through any cold pool or strong precipiating clouds.
Also take a look at the different seasons.


On the other hand Sakat et al showed very wide rang of rain rates “mean rain rates and RWC are between 0.01 - 31 mm d−1” ([Sarkar et al., 2023, p. 12685](zotero://select/library/items/G2B2A8IK)) ([pdf](zotero://open-pdf/library/items/ZNPPEKFT?page=15&annotation=FVB8GLGJ))

In [ ]:
fig, axs = plt.subplots(figsize=(10, 5), ncols=3, sharex=True, sharey=True)
axs[0].scatter(
    surface_precipitation.sel(time=time_slice).mean("time"),
    condensation["mass_represented"]
    .sel(gridbox=condensation["max_gridbox"])
    .sel(time=time_slice)
    .sum("radius_bins")
    .mean("time"),
    marker=".",
    # label=f"cloud {cloud_id.values}",
)
axs[0].set_xlabel(r"Surface precipitation $mm/h$")
axs[0].set_ylabel("LWC [g m$^{-3}$]")
axs[0].set_title("Cloud layer")

axs[1].scatter(
    surface_precipitation.sel(time=time_slice).mean("time"),
    condensation["mass_represented"]
    .sel(gridbox=condensation["max_gridbox"] - 1)
    .sel(time=time_slice)
    .sum("radius_bins")
    .mean("time"),
    marker=".",
    # label=f"cloud {cloud_id.values}",
)
axs[1].set_xlabel(r"Surface precipitation $mm/h$")
axs[1].set_ylabel("Mass [g m$^{-3}$]")
axs[1].set_title("Top sub cloud layer")

axs[2].scatter(
    surface_precipitation.sel(time=time_slice).mean("time"),
    condensation["mass_represented"].sel(gridbox=0).sel(time=time_slice).sum("radius_bins").mean("time"),
    marker=".",
    # label=f"cloud {cloud_id.values}",
)
axs[2].set_xlabel(r"Surface precipitation $mm/h$")
axs[2].set_ylabel("Mass [g m$^{-3}$]")
axs[2].set_title("Surface gridbox")

fig.savefig(fig_dir / "scatter_surface_precipitation_LWC.svg")

Text(0.5, 1.0, 'Surface gridbox')

# Condensation only run as example

#### Cloud height impact

In [ ]:
latent_heating = condensation["latent_heating"]
latent_heating_mean = condensation["latent_heating_mean"]
latent_heating_sum = condensation["latent_heating_sum"]

fig, axs = plt.subplots(ncols=3, figsize=(15, 7), sharey=False)

# Vertical profiles
axs[0].plot(
    latent_heating.where(condensation["sub_cloud_layer_mask"]).T,
    condensation["gridbox_coord3"].T,
    alpha=0.5,
)
axs[0].set_xlabel(label_from_attrs(latent_heating))
axs[0].set_ylabel("Height [m]")
axs[0].set_title("Latent heating profiles")

# Mean evaporation rate
axs[1].scatter(
    latent_heating_mean,
    condensation["cloud_altitude"],
    color="k",
    marker=".",
    alpha=0.5,
)

axs[1].scatter(
    latent_heating_mean.sel(cloud_id=301),
    condensation["cloud_altitude"].sel(cloud_id=301),
    color="r",
    marker="o",
    alpha=0.5,
)
axs[1].set_xlabel(label_from_attrs(latent_heating_mean))
axs[1].set_ylabel("Cloud altitude [m]")
axs[1].set_title("Column mean latent heating")

# axs[0].set_xlim([-4e-5, 0])
# axs[1].set_xlim([-4e-5, 0])


axs[2].scatter(
    latent_heating_sum,
    condensation["cloud_altitude"],
    color="k",
    marker=".",
    alpha=0.5,
)

axs[2].scatter(
    latent_heating_sum.sel(cloud_id=301),
    condensation["cloud_altitude"].sel(cloud_id=301),
    color="r",
    marker="o",
    alpha=0.5,
)
axs[2].set_xlabel(label_from_attrs(latent_heating_sum))
axs[2].set_title("Column integrated latent heating")
# axs[2].set_xlim([-3e-2, 0])

for ax in axs:
    ax.set_yticks([0, 500, 1000])
    ax.set_ylim([0, 1200])

fig.savefig(fig_dir / "latent_heating_profiles.svg")

``NOTE:`` 

**It is good to see, that the evaporation rates are in line with the literature**

*... of 15–352 Wm−2 over a 700 m deep sub-cloud layer is equivalent to 2–50 K d−1 of evaporative cooling. This is comparable to the typical stratocumulus cloud-top radiative longwave cooling (4–10 K d−1) and with the rain evaporation cooling rate at cloud base in the marine sub-cloud stratocumulus deck of 2–20 K d−1 (shown in Wood, 2005).*
([Sarkar et al., 2023, p. 12685](zotero://select/library/items/G2B2A8IK)) ([pdf](zotero://open-pdf/library/items/ZNPPEKFT?page=15&annotation=K6IIGBHX))

We can see that the height of the cloud does alter the mean evaporation rate slightly. 
With more evaporation for lower clouds.
Why this is the case, we do not yet know.
But this can also be due to higher cloud beeing sampled at different days!

In the end there seems to be no big correlation.

BUT: certainly the height does play a big role in terms of total evaporated rate.

#### Thermodynamics

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=2)

sc = axs[0].scatter(
    condensation["latent_heating"],
    condensation["gridbox_coord3"],
    c=condensation["relative_humidity"],
    marker=".",
    alpha=0.8,
    cmap="Blues",
)
fig.colorbar(sc, ax=axs[0], label=label_from_attrs(condensation["relative_humidity"]))

sc = axs[1].scatter(
    condensation["latent_heating"],
    condensation["gridbox_coord3"],
    c=condensation["mass_represented_per_volume"].sum("radius_bins").sel(time=time_slice).mean("time"),
    marker=".",
    alpha=0.8,
    cmap="inferno",
)
fig.colorbar(sc, ax=axs[1], label=label_from_attrs(condensation["mass_represented_per_volume"]))

for ax in axs:
    ax.set_xlabel(label_from_attrs(condensation["latent_heating"]))
    ax.set_ylabel("Height [m]")

fig.savefig(fig_dir / "scatter_latent_heating_relative_humidity_mass.svg")

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=2)

sc = axs[0].scatter(
    condensation["relative_humidity"],
    condensation["gridbox_coord3"],
    c=condensation["latent_heating"],
    marker=".",
    alpha=0.8,
    cmap="inferno",
)
fig.colorbar(sc, ax=axs[0], label=label_from_attrs(condensation["latent_heating"]))
axs[0].set_xlabel(label_from_attrs(condensation["relative_humidity"]))

sc = axs[1].scatter(
    condensation["mass_represented_per_volume"].sum("radius_bins").sel(time=time_slice).mean("time"),
    condensation["gridbox_coord3"],
    c=condensation["latent_heating"],
    marker=".",
    alpha=0.8,
    cmap="inferno",
)
fig.colorbar(sc, ax=axs[1], label=label_from_attrs(condensation["latent_heating"]))
axs[1].set_xlabel(label_from_attrs(condensation["mass_represented_per_volume"]))

for ax in axs:
    ax.set_ylabel("Height [m]")

fig.suptitle("Relative humidity, Mass in gridbox vs. Evaporation energy rate")

fig.savefig(fig_dir / "scatter_relative_humidity_mass_latent_heating.svg")

In [ ]:
data = condensation

fig, ax = plt.subplots(figsize=(10, 4.5))

sc = ax.scatter(
    data["mass_represented_per_volume"].sum("radius_bins").sel(time=time_slice).mean("time"),
    data["relative_humidity"],
    c=1e3 * data["latent_heating"],
    marker=".",
    alpha=0.8,
    cmap="inferno_r",
)

ax.set_xlabel(label_from_attrs(data["mass_represented_per_volume"]))
ax.set_ylabel(label_from_attrs(data["relative_humidity"]))
fig.colorbar(sc, ax=ax, label=label_from_attrs(data["latent_heating"]))

fig.suptitle("Evaporation energy, Mass in gridbox vs. relative humidity")

fig.savefig(fig_dir / "latent_heating_against_mass_humidity.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

ax.scatter(
    condensation["relative_humidity"],
    condensation["latent_heating"],
    marker=".",
    alpha=0.5,
)
ax.set_xlabel(label_from_attrs(condensation["relative_humidity"]))
ax.set_ylabel(label_from_attrs(condensation["latent_heating"]))
fig.suptitle("Latent heating rate vs. relative humidity")
fig.savefig(fig_dir / "latent_heating_against_humidity.svg")
fig, ax = plt.subplots(figsize=(7, 5))

ax.scatter(
    condensation["mass_represented_per_volume"].sum("radius_bins").sel(time=time_slice).mean("time"),
    condensation["latent_heating"],
    marker=".",
    alpha=0.5,
)
ax.set_xlabel(label_from_attrs(condensation["mass_represented_per_volume"]))
ax.set_ylabel(label_from_attrs(condensation["latent_heating"]))
fig.suptitle("Latent heating rate vs. mass")
fig.savefig(fig_dir / "latent_heating_against_mass.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))


latent_heating_anomaly = condensation["latent_heating"] - condensation["latent_heating"].mean(
    "gridbox", keep_attrs=True
)
latent_heating_anomaly.attrs["long_name"] = "Latent heating anomaly"
latent_heating_anomaly.attrs["units"] = condensation["latent_heating"].attrs["units"]

ax.scatter(
    condensation["relative_humidity"],
    latent_heating_anomaly,
    marker=".",
    alpha=0.5,
)
ax.set_xlabel(label_from_attrs(condensation["relative_humidity"]))
ax.set_ylabel(label_from_attrs(latent_heating_anomaly))
fig.suptitle("Latent heating anomaly vs. relative humidity")
fig.savefig(fig_dir / "latent_heating_anomaly_against_humidity.svg")
fig, ax = plt.subplots(figsize=(7, 5))

ax.scatter(
    condensation["mass_represented_per_volume"].sum("radius_bins").sel(time=time_slice).mean("time"),
    latent_heating_anomaly,
    marker=".",
    alpha=0.5,
)
ax.set_xlabel(label_from_attrs(condensation["mass_represented_per_volume"]))
ax.set_ylabel(label_from_attrs(latent_heating_anomaly))
fig.suptitle("Latent heating anomaly vs. mass")
fig.savefig(fig_dir / "latent_heating_anomaly_against_mass.svg")

# Microphysics

### Vertical profiles of evaporation rate

In [ ]:
var = "latent_heating"


datas = [condensation, collision_condensation, coalbure_condensation_large, coalbure_condensation_small]
data_colors = default_colors[: len(datas)]

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=True, width_ratios=[2, 1, 1, 1, 1]
)

idx = 0
for data, color in zip(datas, data_colors):
    idx += 1
    evaporation = data["latent_heating"]
    axs[idx].plot(1e3 * evaporation.T, data["gridbox_coord3"].T, color=color, linestyle="-", alpha=0.1)

    axs[idx].plot(
        1e3 * evaporation.median("cloud_id").T,
        data["gridbox_coord3"].median("cloud_id").T,
        color=color,
        linestyle="-",
        alpha=1,
    )

    axs[0].plot(
        1e3 * evaporation.median("cloud_id").T,
        data["gridbox_coord3"].median("cloud_id").T,
        color=color,
        linestyle="-",
        alpha=1,
        label=data.attrs["microphysics"],
    )
    axs[idx].set_title(data.attrs["microphysics"])

axs[0].axvline(0, color="black", linestyle="--", alpha=0.5)
axs[0].set_title("Difference of all microphysics")
axs[0].set_ylabel("Height [m]")
# axs[0].legend(fontsize = 10)

for ax in axs:
    # ax.set_xlabel(f"Evaporation rate in $\\left[m{evaporation.attrs['units']}\\right]$")
    ax.set_xlim(-100, 20)
    ax.set_xticks([-75, -50, -25])
fig.suptitle("Evaporation rate profiles for different microphysics")  #
# fig.tight_layout()

Text(0.5, 0.98, 'Evaporation rate profiles for different microphysics')

In [ ]:
var = "latent_heating"


datas = [condensation, collision_condensation, coalbure_condensation_large, coalbure_condensation_small]
data_colors = default_colors[: len(datas)]

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=False, width_ratios=[2, 1, 1, 1, 1]
)

idx = 0
for data, color in zip(datas, data_colors):
    idx += 1
    select_ids = (data["max_gridbox"] <= 33) & (data["max_gridbox"] >= 33)
    data = data.sel(cloud_id=select_ids)
    evaporation = data["latent_heating"] * 1e3
    evaporation.attrs["units"] = "mW m$^{-3}$"
    axs[idx].plot(evaporation.T, data["gridbox_coord3"].T, color=color, linestyle="-", alpha=0.3)

    axs[idx].plot(
        evaporation.median("cloud_id").T,
        data["gridbox_coord3"].median("cloud_id").T,
        color=color,
        linestyle="-",
        linewidth=2,
        alpha=1,
    )

    axs[0].plot(
        evaporation.median("cloud_id").T,
        data["gridbox_coord3"].median("cloud_id").T,
        color=color,
        linestyle="-",
        linewidth=2,
        alpha=1,
        label=data.attrs["microphysics"],
    )
    axs[idx].set_title(data.attrs["microphysics"])

axs[0].axvline(0, color="black", linestyle="--", alpha=0.5)
axs[0].set_title("Difference of all microphysics")
axs[0].set_ylabel("Height [m]")
axs[0].legend(loc="upper left")

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.set_xlim(-100, 0)
    ax.set_xticks([-75, -50, -25])
axs[0].set_xlim(-65, -35)
axs[0].set_xticks([-60, -50, -40])
fig.suptitle("Latent heating for different microphysics")  #
fig.tight_layout()
fig.savefig(fig_dir / "microphysics_latent_heating_profiles.svg")

### Difference between ``Condensation`` and other microphysics

only gridbox betweenm 30 and 40

**Percentage of evaporation compared to condensation**

In [ ]:
var = "latent_heating"

datas = [collision_condensation, coalbure_condensation_large, coalbure_condensation_small]
data_colors = default_colors[1 : len(datas) + 1]

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=True, width_ratios=[2, 1, 1, 1]
)

idx = 0
for data, color in zip(datas, data_colors):
    idx += 1
    select_ids = (data["max_gridbox"] <= 33) & (data["max_gridbox"] >= 33)
    data = data.sel(cloud_id=select_ids)
    diff = (
        (data[var] / condensation[var])
        * 100
        # - (data['evaporation'].sel(gridbox = 0) - condensation['evaporation'].sel(gridbox = 0))
    )
    diff.attrs["units"] = "\\%"
    diff.attrs["long_name"] = "Lantent heating fraction\ncompared to condensation only"
    data = data.sel(cloud_id=diff["cloud_id"])
    axs[idx].plot(diff.T, data["gridbox_coord3"].T, color=color, linestyle="-", alpha=0.3)

    axs[idx].plot(
        diff.median("cloud_id").T,
        data["gridbox_coord3"].median("cloud_id").T,
        color=color,
        linestyle="-",
        linewidth=2,
        alpha=1,
    )
    axs[0].plot(
        diff.median("cloud_id").T,
        data["gridbox_coord3"].median("cloud_id").T,
        color=color,
        linestyle="-",
        linewidth=2,
        alpha=1,
        label=data.attrs["microphysics"],
    )
    axs[idx].set_title(data.attrs["microphysics"])


data = condensation
select_ids = (data["max_gridbox"] <= 34) & (data["max_gridbox"] >= 30)
data = data.sel(cloud_id=select_ids)
diff = (
    (data[var] / condensation[var])
    * 100
    # - (data['evaporation'].sel(gridbox = 0) - condensation['evaporation'].sel(gridbox = 0))
)
diff.attrs["units"] = "\\%"
diff.attrs["long_name"] = "Lantent heating fraction\ncompared to condensation only"

axs[0].plot(
    diff.median("cloud_id").T,
    data["gridbox_coord3"].median("cloud_id").T,
    color=default_colors[0],
    linestyle="-",
    linewidth=2,
    alpha=1,
    label=data.attrs["microphysics"],
)

axs[0].set_title("Difference of all microphysics")
axs[0].set_ylabel("Height [m]")
axs[0].legend(loc="upper left")

for ax in axs:
    ax.axvline(100, color="black", linestyle="--", alpha=0.5)
    ax.set_xlabel(label_from_attrs(diff))
    ax.set_xlim(90, 102)
    ax.set_xticks([90, 95, 100])
fig.suptitle("Fraction of evaporation rate to condensation only")
fig.tight_layout()
fig.savefig(fig_dir / "microphysics_latent_heating_fraction_profiles.svg")

## Impact of Droplet size

In [ ]:
cloud_ids_from_gridbox_33 = condensation["cloud_id"].where(
    condensation["max_gridbox"].compute() == 33, drop=True
)

datas = [
    null_microphysics,
    condensation,
    collision_condensation,
    coalbure_condensation_large,
    coalbure_condensation_small,
]

latent_heat_comparison = dict()
multiplicity_comparison = dict()
number_superdroplets_comparison = dict()

for data in datas:
    mp = data.attrs["microphysics_short"]
    da = (
        1e3
        * data["latent_heating_radius_bins"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_ids_from_gridbox_33)
        .median("time", keep_attrs=True)
        .reindex(radius_bins=combined_radius_bins, fill_value=0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "$mW m^{-3}$"
    da.attrs["long_name"] = "Latent heating"
    latent_heat_comparison[mp] = da

for data in datas:
    mp = data.attrs["microphysics_short"]

    da = (
        8
        * data["xi_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_ids_from_gridbox_33)
        .mean("time", keep_attrs=True)
        .reindex(radius_bins=combined_radius_bins, fill_value=0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "$ 8000 m^{-3}$"
    da.attrs["long_name"] = "Multiplicity of SDs"

    multiplicity_comparison[mp] = da

for data in datas:
    mp = data.attrs["microphysics_short"]

    da = (
        8
        * data["number_superdroplets_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_ids_from_gridbox_33)
        .mean("time", keep_attrs=True)
        .reindex(radius_bins=combined_radius_bins, fill_value=0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "$\\#/8000 m^{-3}$"
    da.attrs["long_name"] = "Number of SDs"

    number_superdroplets_comparison[mp] = da

In [ ]:
keys = [
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

fig, axs = plt.subplots(ncols=len(keys) + 1, nrows=4, figsize=(20, 10), sharex=False, sharey=True)

axs_absolute = axs[0, :]
axs_anomalie = axs[1, :]


for idx, mp in enumerate(keys):
    data = latent_heat_comparison[mp]
    data = data  # .where(data != 0)
    diff = data - latent_heat_comparison["condensation"]
    diff.attrs["units"] = data.attrs["units"]
    diff.attrs["long_name"] = f"Latent heating difference"

    pcm1 = axs[0, idx].pcolormesh(
        data["radius_bins"],
        data["gridbox"] * 20,
        data.mean("cloud_id"),
        shading="nearest",
        cmap="inferno",
        vmin=-10,
        vmax=0,
    )
    fig.colorbar(mappable=pcm1, label=label_from_attrs(data))

    pcm2 = axs[1, idx].pcolormesh(
        diff["radius_bins"],
        diff["gridbox"] * 20,
        diff.mean("cloud_id"),
        shading="nearest",
        cmap="RdBu_r",
        vmin=-1.5,
        vmax=1.5,
    )
    fig.colorbar(mappable=pcm2, label=label_from_attrs(data))

    nbr = multiplicity_comparison[mp]
    nbr = nbr.where(nbr <= 1e10)

    pcm3 = axs[2, idx].pcolormesh(
        nbr["radius_bins"],
        nbr["gridbox"] * 20,
        (nbr - multiplicity_comparison["condensation"]).mean("cloud_id"),
        shading="nearest",
        cmap="RdBu_r",
        norm=mcolors.SymLogNorm(linthresh=1e-1, linscale=1e-6, vmin=-1e3, vmax=1e3),
    )
    fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

    nbr = number_superdroplets_comparison[mp]

    pcm3 = axs[3, idx].pcolormesh(
        nbr["radius_bins"],
        nbr["gridbox"] * 20,
        (nbr - number_superdroplets_comparison["condensation"]).mean("cloud_id"),
        shading="nearest",
        cmap="RdBu_r",
        vmin=-1,
        vmax=1,
    )
    fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

    axs[0, -1].plot(data.sum("radius_bins").mean("cloud_id"), data["gridbox"] * 20, label=mp)

    axs[1, -1].plot(diff.sum("radius_bins").mean("cloud_id"), diff["gridbox"] * 20, label=mp)

    axs[0, idx].set_title(data_dict[mp]["microphysics"])


axs[0, -1].set_title("Sum over all radius bins")


for ax in axs[:, :-1].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xlim(None, None)

for ax in axs[:, :-1].flatten():
    ax.set_xscale("log")

for ax in axs.flatten():
    ax.set_ylim(0, 650)


axs[0, 0].set_ylabel("Latent heating\nAbsolute values\n\nHeight [m]")
axs[1, 0].set_ylabel("Latent heating\nDiff to condensation\n\nHeight [m]")
axs[2, 0].set_ylabel("Multiplicity\n\n\nHeight [m]")
axs[3, 0].set_ylabel("Number SDs\n\n\nHeight [m]")

fig.tight_layout()
fig.savefig(fig_dir / "latent_heating_comparison_mean.png", dpi=400)

In [ ]:
keys = [
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

fig, axs = plt.subplots(ncols=len(keys) + 1, nrows=4, figsize=(20, 10), sharex=False, sharey=True)

axs_absolute = axs[0, :]
axs_anomalie = axs[1, :]


for idx, mp in enumerate(keys):
    data = latent_heat_comparison[mp]
    data = data  # .where(data != 0)
    diff = data - latent_heat_comparison["condensation"]
    diff.attrs["units"] = data.attrs["units"]
    diff.attrs["long_name"] = f"Latent heating difference"

    pcm1 = axs[0, idx].pcolormesh(
        data["radius_bins"],
        data["gridbox"] * 20,
        data.median("cloud_id"),
        shading="nearest",
        cmap="inferno",
        vmin=-10,
        vmax=0,
    )
    fig.colorbar(mappable=pcm1, label=label_from_attrs(data))

    pcm2 = axs[1, idx].pcolormesh(
        diff["radius_bins"],
        diff["gridbox"] * 20,
        diff.median("cloud_id"),
        shading="nearest",
        cmap="RdBu_r",
        vmin=-1.5,
        vmax=1.5,
    )
    fig.colorbar(mappable=pcm2, label=label_from_attrs(data))

    nbr = multiplicity_comparison[mp]
    nbr = nbr.where(nbr <= 1e10)

    pcm3 = axs[2, idx].pcolormesh(
        nbr["radius_bins"],
        nbr["gridbox"] * 20,
        (nbr - multiplicity_comparison["condensation"]).median("cloud_id"),
        shading="nearest",
        cmap="RdBu_r",
        norm=mcolors.SymLogNorm(linthresh=1e-3, linscale=1e-6, vmin=-1e3, vmax=1e3),
    )
    fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

    nbr = number_superdroplets_comparison[mp]

    pcm3 = axs[3, idx].pcolormesh(
        nbr["radius_bins"],
        nbr["gridbox"] * 20,
        (nbr - number_superdroplets_comparison["condensation"]).median("cloud_id"),
        shading="nearest",
        cmap="RdBu_r",
        vmin=-1,
        vmax=1,
    )
    fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

    axs[0, -1].plot(data.sum("radius_bins").median("cloud_id"), data["gridbox"] * 20, label=mp)

    axs[1, -1].plot(diff.sum("radius_bins").median("cloud_id"), diff["gridbox"] * 20, label=mp)

    axs[0, idx].set_title(data_dict[mp]["microphysics"])


axs[0, -1].set_title("Sum over all radius bins")


for ax in axs[:, :-1].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xlim(None, None)

for ax in axs[:, :-1].flatten():
    ax.set_xscale("log")

for ax in axs.flatten():
    ax.set_ylim(0, 650)


axs[0, 0].set_ylabel("Latent heating\nAbsolute values\n\nHeight [m]")
axs[1, 0].set_ylabel("Latent heating\nDiff to condensation\n\nHeight [m]")
axs[2, 0].set_ylabel("Multiplicity\n\n\nHeight [m]")
axs[3, 0].set_ylabel("Number SDs\n\n\nHeight [m]")


fig.tight_layout()
fig.savefig(fig_dir / "latent_heating_comparison_median.png", dpi=400)

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/dask/utils.py:73: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/dask/utils.py:73: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/dask/utils.py:73: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/dask/utils.py:73: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)


In [ ]:
cloud_ids = [222, 142, 301, 18]

# data = coalbure_condensation_large

fig, axs = plt.subplots(
    nrows=3, ncols=5, figsize=(20, 6), sharex=True, sharey=False, width_ratios=[1, 1, 1, 1, 0.02]
)

for idx, cloud_id in enumerate(cloud_ids):
    ds = coalbure_condensation_large.sel(cloud_id=cloud_id).sel(time=time_slice)
    dm = ds["domain_mask"].fillna(0).astype(bool).compute()
    ds = ds.where(dm)
    ds["max_gridbox"] = ds["max_gridbox"].mean().compute()

    data = 1e3 * ds["latent_heating_radius_bins"]

    ax = axs[0, idx]
    data.median("time").plot(
        ax=ax,
        cmap="inferno",
        vmin=-12,
        vmax=0,
        cbar_kwargs=dict(label="Evaporation [mW m$^{-3}$]", cax=axs[0, -1]),
    )
    ax.set_title(f"Cloud ID: {cloud_id}\nEvaporation rate [mW m$^{-3}$]")
    ax.set_ylim(0, 40)

    ax = axs[1, idx]
    (data.median("time") - data.median("time").mean("gridbox")).plot(
        ax=ax, cmap="RdBu", vmin=-4, vmax=4, cbar_kwargs=dict(label="Anomaly evap. [mW m$^{-3}$]")
    )
    ax.set_title(f"Anomaly of evap. rate [mW m$^{-3}$]")

    ax.set_xscale("log")
    ax.set_xlabel("Radius [µm]")
    ax.set_ylim(0, 40)

    ax = axs[2, idx]

    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=ds["max_gridbox"]).median("time"),
        color="k",
        linestyle="-",
        alpha=0.5,
        label="Cloud layer",
    )
    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=ds["max_gridbox"] - 1).median("time"),
        color="b",
        linestyle="-",
        alpha=0.5,
        label="Top Sub Cloud layer",
    )

    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=0).median("time"),
        color="r",
        linestyle="-",
        alpha=0.5,
        label="Surface layer",
    )
    ax.legend(fontsize=8)
    ax.set_yscale("log")

ax.set_xlim(10e0, None)

fig.tight_layout()
fig.savefig(fig_dir / "evaporation_rate_anomaly_clouds.svg")

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)


/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)


In [ ]:
cloud_ids = [222, 142, 301, 18]


fig, axs = plt.subplots(
    nrows=3, ncols=5, figsize=(20, 6), sharex=True, sharey=False, width_ratios=[1, 1, 1, 1, 0.02]
)

for idx, cloud_id in enumerate(cloud_ids):
    ds = condensation.sel(cloud_id=cloud_id).sel(time=time_slice)
    data = 1e3 * ds["latent_heating_radius_bins"]
    ax = axs[0, idx]
    data.mean("time").plot(
        ax=ax,
        cmap="inferno",
        vmin=-12,
        vmax=0,
        cbar_kwargs=dict(label="Evaporation [mW m$^{-3}$]", cax=axs[0, -1]),
    )
    ax.set_title(f"Cloud ID: {cloud_id}\nEvaporation rate [mW m$^{-3}$]")
    ax.set_ylim(0, 40)

    ax = axs[1, idx]
    (data - data.mean("gridbox")).sel(time=time_slice).mean("time").plot(
        ax=ax, cmap="RdBu", vmin=-4, vmax=4, cbar_kwargs=dict(label="Anomaly evap. [mW m$^{-3}$]")
    )
    ax.set_title(f"Anomaly of evap. rate [mW m$^{-3}$]")

    ax.set_xscale("log")
    ax.set_xlabel("Radius [µm]")
    ax.set_ylim(0, 40)

    ax = axs[2, idx]

    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=ds["max_gridbox"]).median("time"),
        color="k",
        linestyle="-",
        alpha=0.5,
        label="Cloud layer",
    )
    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=ds["max_gridbox"] - 1).median("time"),
        color="b",
        linestyle="-",
        alpha=0.5,
        label="Top Sub Cloud layer",
    )

    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=0).median("time"),
        color="r",
        linestyle="-",
        alpha=0.5,
        label="Surface layer",
    )
    ax.legend(fontsize=8)
    ax.set_yscale("log")

ax.set_xlim(10e0, None)

fig.tight_layout()
fig.savefig(fig_dir / "evaporation_rate_anomaly_clouds.svg")

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)


/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)


In [ ]:
cloud_ids = [222, 142, 301, 18]

fig, axs = plt.subplots(
    nrows=3, ncols=5, figsize=(20, 6), sharex=True, sharey=False, width_ratios=[1, 1, 1, 1, 0.02]
)

for idx, cloud_id in enumerate(cloud_ids):
    ds = condensation.sel(cloud_id=cloud_id).sel(time=time_slice)
    data = 1e3 * ds["latent_heating_radius_bins"]
    ax = axs[0, idx]
    data.median("time").plot(
        ax=ax,
        cmap="inferno",
        vmin=-12,
        vmax=0,
        cbar_kwargs=dict(label="Evaporation [mW m$^{-3}$]", cax=axs[0, -1]),
    )
    ax.set_title(f"Cloud ID: {cloud_id}\nEvaporation rate [mW m$^{-3}$]")
    ax.set_ylim(0, 40)

    ax = axs[1, idx]
    (data.median("time") - data.median("time").mean("gridbox")).plot(
        ax=ax, cmap="RdBu", vmin=-4, vmax=4, cbar_kwargs=dict(label="Anomaly evap. [mW m$^{-3}$]")
    )
    ax.set_title(f"Anomaly of evap. rate [mW m$^{-3}$]")

    ax.set_xscale("log")
    ax.set_xlabel("Radius [µm]")
    ax.set_ylim(0, 40)

    ax = axs[2, idx]

    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=ds["max_gridbox"]).median("time"),
        color="k",
        linestyle="-",
        alpha=0.5,
        label="Cloud layer",
    )
    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=ds["max_gridbox"] - 1).median("time"),
        color="b",
        linestyle="-",
        alpha=0.5,
        label="Top Sub Cloud layer",
    )

    ax.plot(
        ds["radius_bins"],
        ds["xi"].sel(gridbox=0).median("time"),
        color="r",
        linestyle="-",
        alpha=0.5,
        label="Surface layer",
    )
    ax.legend(fontsize=8)
    ax.set_yscale("log")

ax.set_xlim(10e0, None)

fig.tight_layout()
fig.savefig(fig_dir / "evaporation_rate_anomaly_clouds.svg")

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)
/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/xarray/plot/utils.py:724: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  cbar = fig.colorbar(primitive, **cbar_kwargs)


# Temporal evolution of the distributions

In [ ]:
def temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))
    fig, axs = plt.subplots(
        ncols=2, nrows=1, figsize=(15, 7), sharex=False, sharey=True, width_ratios=[1, 0.5]
    )

    try:
        if "$" in ds[variable].attrs["units"]:
            clabel = f"{ds[variable].attrs['long_name']} {ds[variable].attrs['units']}"
        else:
            clabel = f"{ds[variable].attrs['long_name']} ${ds[variable].attrs['units']}$"
    except:
        if "$" in ds[variable].attrs["units"]:
            clabel = f"{variable} {ds[variable].attrs['units']}"
        else:
            clabel = f"{variable} ${ds[variable].attrs['units']}$"

    pcm = axs[0].pcolormesh(
        ds[variable]["time"],
        ds["gridbox_coord3"],
        ds[variable].sum("radius_bins").T,
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )

    fig.colorbar(mappable=pcm, ax=axs[0], label=clabel, extend="both")
    axs[0].set_xlabel("Time $s$")
    axs[0].set_ylabel("Height $m$")

    ds_stationary = ds.sel(time=slice(400, None))

    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").quantile(0.5, "time"),
        ds_stationary["gridbox_coord3"],
    )
    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").T,
        ds_stationary["gridbox_coord3"],
        color=[0.7, 0.7, 0.7],
        linewidth=0.1,
        alpha=0.1,
        zorder=0,
    )
    axs[1].fill_betweenx(
        ds_stationary["gridbox_coord3"],
        ds_stationary[variable].sum("radius_bins").quantile(0.25, "time"),
        ds_stationary[variable].sum("radius_bins").quantile(0.75, "time"),
        color="k",
        alpha=0.1,
    )
    axs[1].set_xlim(vmin, vmax)
    axs[1].set_xlabel(clabel)

    axs[1].set_ylabel("Height $m$")

    fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

    return fig, axs


def temporal_evolution_of_variable_dataarray(ds, variable, cloud_id, vmin, vmax):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    fig, axs = plt.subplots(
        ncols=2, nrows=1, figsize=(15, 7), sharex=False, sharey=True, width_ratios=[1, 0.5]
    )

    pcm = axs[0].pcolormesh(
        ds["time"],
        ds["gridbox_coord3"],
        ds[variable].T,
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )
    fig.colorbar(
        mappable=pcm, ax=axs[0], label=f"{variable} {ds[variable].attrs['units']}", extend="both"
    )
    axs[0].set_xlabel("Time $s$")
    axs[0].set_ylabel("Height $m$")

    ds_stationary = ds.sel(time=slice(1500, None))

    axs[1].plot(
        ds_stationary[variable].quantile(0.5, "time"),
        ds_stationary["gridbox_coord3"],
    )
    axs[1].plot(
        ds_stationary[variable].T,
        ds_stationary["gridbox_coord3"],
        color=[0.7, 0.7, 0.7],
        linewidth=0.1,
        alpha=0.1,
        zorder=0,
    )
    axs[1].fill_betweenx(
        ds_stationary["gridbox_coord3"],
        ds_stationary[variable].quantile(0.25, "time"),
        ds_stationary[variable].quantile(0.75, "time"),
        color="k",
        alpha=0.1,
    )
    axs[1].set_xlim(vmin, vmax)
    axs[1].set_xlabel(f"{variable} {ds[variable].attrs['units']}")
    axs[1].set_ylabel("Height $m$")

    fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

    return fig, axs


def temporal_evolution_different_gridboxes(
    data_dict, key, variable, cloud_id, vmin, vmax, isel_gridboxes=[-1, -2, -3, -5, 10, 0]
):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))

    try:
        if "$" in ds[variable].attrs["units"]:
            clabel = f"{ds[variable].attrs['long_name']} {ds[variable].attrs['units']}"
        else:
            clabel = f"{ds[variable].attrs['long_name']} ${ds[variable].attrs['units']}$"
    except:
        if "$" in ds[variable].attrs["units"]:
            clabel = f"{variable} {ds[variable].attrs['units']}"
        else:
            clabel = f"{variable} ${ds[variable].attrs['units']}$"

    gridboxes = ds["gridbox"].isel(gridbox=isel_gridboxes).compute()
    fig, axs = plt.subplots(
        ncols=len(gridboxes) + 1, nrows=1, figsize=(15, 10), sharex=True, sharey=True
    )

    for idx, gridbox in enumerate(gridboxes):
        altitude = ds["gridbox_coord3"].sel(gridbox=gridbox).compute()
        ds_selected = ds.sel(gridbox=gridbox)
        pcm = axs[idx].pcolormesh(
            ds_selected[variable]["radius_bins"],
            ds_selected[variable]["time"],
            ds_selected[variable],
            shading="auto",
            cmap=my_cmap,
            norm=my_norm,
        )
        axs[idx].set_title(f"Gridbox {gridbox.data}\n{altitude.data}m")
    for ax in axs:
        ax.set_xticks([1e0, 1e1, 1e2, 1e3])
        ax.set_xlim(1e0, 1e3)
        ax.grid()
        ax.set_xlabel("Radius $\\mu m$")
    axs[0].set_xscale("log")

    axs[0].set_ylabel("Time $s$")

    fig.colorbar(
        ax=axs[-1],
        mappable=pcm,
        orientation="vertical",
        label=f"{variable} {ds[variable].attrs['units']}",
        extend="both",
    )
    fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

    return fig, axs

#### Multiplicity

In [ ]:
vmin, vmax = 1e-10, 0.2
variable = "mass_represented_per_volume"
cloud_id = 222
key = "coalbure_condensation_large"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, 0.03)

In [ ]:
vmin, vmax = 1e-10, 10e6
variable = "xi"
cloud_id = 222
key = "condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, 0.3e6)

In [ ]:
vmin, vmax = 1e-10, 0.03
variable = "mass_represented_per_volume"
cloud_id = 222
key = "condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)

In [ ]:
vmin, vmax = -0.02, -1e-10
variable = "latent_heating_radius_bins"
cloud_id = 222
key = "condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)

In [ ]:
vmin, vmax = -0.02, -1e-10
variable = "latent_heating_radius_bins"
cloud_id = 222
key = "collision_condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)

In [ ]:
vmin, vmax = 1e-10, 40
variable = "number_superdroplets_per_volume"
cloud_id = 222
key = "condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, 3)

NameError: name 'temporal_evolution_different_gridboxes' is not defined

# More stuff

In [ ]:
def plot_single_microphysics(ax, data_dict, data_keys, variable, cloud_dict):
    ax_microlabel = ax.twinx()
    ax_microlabel.set_yticks([])

    plot_cloud_lines = True
    for key in data_keys:
        dataset = data_dict[key]["dataset"]
        linestyle = data_dict[key]["linestyle"]
        color = data_dict[key]["color"]
        dataset = dataset.sel(cloud_id=dataset["max_gridbox"] < 40)
        dataset = dataset.sel(time=slice(1500, None))

        for ID in cloud_dict:
            cloud_id = cloud_dict[ID]["cloud_id"]
            ax.plot(
                dataset.sel(cloud_id=cloud_id)[variable],
                dataset["pseudo_coord3"],
                color=cloud_dict[ID]["color"],
                alpha=1,
                linestyle=linestyle,
                zorder=3,
            )
        if plot_cloud_lines == True:
            for ID in cloud_dict:
                ax.plot(
                    np.nan,
                    np.nan,
                    color=cloud_dict[ID]["color"],
                    alpha=1,
                    label=f"cloud {ID}",
                    linestyle="-",
                    zorder=3,
                )
            plot_cloud_lines = False

        ax_microlabel.plot(
            np.nan,
            np.nan,
            color="k",
            alpha=0,
            label=f"{data_dict[key]['microphysics']}",
            linestyle=linestyle,
        )

        ax.plot(
            dataset[variable].quantile(0.5, "cloud_id"),
            dataset["pseudo_coord3"],
            color="k",
            alpha=1,
            linestyle=linestyle,
            zorder=1,
        )
        ax.fill_betweenx(
            dataset["pseudo_coord3"],
            dataset[variable].quantile(0.25, "cloud_id"),
            dataset[variable].quantile(0.75, "cloud_id"),
            color="k",
            alpha=0.1,
            linestyle=linestyle,
            zorder=1,
        )

    xlabel = dataset[variable].long_name + f" {dataset[variable].attrs['units']}"
    ax.set_xlabel(xlabel)

    # # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

    set_yticks_height(ax)
    ax.set_ylim(0, 800)
    ax.set_ylabel("Height $m$")

    ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
    ax.legend(handler_map=handler_map_alpha(), loc="upper right")

    return ax


def plot_single_microphysics_all(ax, data_dict, data_keys, variable, cloud_dict):
    ax_microlabel = ax.twinx()
    ax_microlabel.set_yticks([])

    plot_cloud_lines = True
    for key in data_keys:
        dataset = data_dict[key]["dataset"]
        linestyle = data_dict[key]["linestyle"]
        color = data_dict[key]["color"]
        dataset = dataset.sel(cloud_id=dataset["max_gridbox"] < 40)
        dataset = dataset.sel(time=slice(1500, None))

        for ID in cloud_dict:
            cloud_id = cloud_dict[ID]["cloud_id"]
            ax.plot(
                dataset.sel(cloud_id=cloud_id)[variable],
                dataset["pseudo_coord3"],
                color=cloud_dict[ID]["color"],
                alpha=1,
                linestyle=linestyle,
                zorder=3,
            )
        if plot_cloud_lines == True:
            for ID in cloud_dict:
                ax.plot(
                    np.nan,
                    np.nan,
                    color=cloud_dict[ID]["color"],
                    alpha=1,
                    label=f"cloud {ID}",
                    linestyle="-",
                    zorder=3,
                )
            plot_cloud_lines = False

        ax_microlabel.plot(
            np.nan,
            np.nan,
            color="k",
            alpha=0,
            label=f"{data_dict[key]['microphysics']}",
            linestyle=linestyle,
        )

        ax.plot(
            dataset[variable].quantile(0.5, "cloud_id"),
            dataset["pseudo_coord3"],
            color="k",
            alpha=1,
            linestyle=linestyle,
            zorder=1,
        )
        ax.plot(
            dataset[variable].T,
            dataset["pseudo_coord3"],
            color=[0.3, 0.3, 0.3],
            alpha=0.4,
            linestyle="-",
            marker=".",
            linewidth=0.3,
            zorder=1,
        )

    xlabel = dataset[variable].long_name + f" {dataset[variable].attrs['units']}"
    ax.set_xlabel(xlabel)

    # # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

    set_yticks_height(ax)
    ax.set_ylim(0, 800)
    ax.set_ylabel("Height $m$")

    ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
    ax.legend(handler_map=handler_map_alpha(), loc="upper right")

    return ax

### Vertical profiles of mass fraction

In [ ]:
variable = "vertical_profile_mass_fraction"
keys = ["null_microphysics", "condensation", "collision_condensation", "coalbure_condensation_large"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks([80, 90, 100])
ax.set_xlim(80, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")


fig.savefig(fig_dir / "mass_fraction.svg", transparent=True)

/work/mh1126/m301096/conda/envs/sdm_pysd_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [ ]:
plt.imshow(dataset["liquid_water_content"].sum("radius_bins", skipna=False).mean("time", skipna=False).T)
plt.imshow(dataset["pseudo_gribox_volume_full"], cmap="Reds", alpha=0.5)

In [ ]:
time_slice = slice(1500, 3590)
units_conversion = 1 / (20**3) * 1e3  # from kg s-1 to g m-3 s-1
monitor_conversion = 1 / (20**3) * 1 / 2 * 1e18  # from g      to g m-3 s-1

zarr = xr.open_dataset(
    f"/home/m/m301096/CLEO/data/output_v3.5/{microphysic}/clusters_{cloud_id}/eurec4a1d_sol.zarr",
    engine="zarr",
    consolidated=False,
)
zarr = zarr.sel(time=time_slice)
direct = zarr["massdelta_cond"].isel(gbxindex=slice(1, -2)) * monitor_conversion

In [ ]:
variable = "mass_difference_timestep"
keys = ["collision_condensation", "coalbure_condensation_large", "null_microphysics", "condensation"]
colors = ["black", "red", "blue", "green"]
time_slice = slice(2500, None)

units_conversion = 1 / (20**3) * 1 / 2 * 1e3  # from kg s-1 to g m-3 s-1
monitor_conversion = 1 / (20**3) * 1 / 2 * 1e18  # from g      to g m-3 s-1


fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

for microphysic, color in zip(keys, colors):
    dataset = data_dict[microphysic]["dataset"].sel(time=time_slice)

    for cloud_id in [222, 142, 301]:
        zarr = xr.open_dataset(
            f"/home/m/m301096/CLEO/data/output_v3.5/{microphysic}/clusters_{cloud_id}/eurec4a1d_sol.zarr",
            engine="zarr",
            consolidated=False,
        )
        zarr = zarr.sel(time=time_slice)
        direct = (
            zarr["massdelta_cond"].sel(
                gbxindex=slice(1, dataset["max_gridbox"].sel(cloud_id=cloud_id) - 1)
            )
            * monitor_conversion
        )

        direct_mean = direct.mean("time")
        direct_std_of_mean = direct.std("time") / np.sqrt(direct["time"].size)
        sub_cloud_mask = (dataset["gridbox"] < dataset["max_gridbox"].sel(cloud_id=cloud_id)) & (
            dataset["gridbox"] > 0
        )
        coord3 = dataset["pseudo_coord3_full"].sel(cloud_id=cloud_id)

        direct_coord3 = coord3.sel(gridbox=slice(direct["gbxindex"].min(), direct["gbxindex"].max()))

        var_profile = units_conversion * dataset[variable].sel(cloud_id=cloud_id).sum(
            "radius_bins"
        ).where(sub_cloud_mask)

        var_profile_mean = var_profile.median("time", skipna=False)
        var_profile_std_of_mean = var_profile.std("time", skipna=False) / np.sqrt(
            var_profile["time"].size
        )

        # axs[0].plot(
        #     var_profile.T,
        #     coord3,
        #     color=color,
        #     alpha=0.01,
        #     linestyle="",
        #     marker='.',
        #     zorder=1,

        # )
        # for ax in axs:
        # ax.plot(
        #     var_profile_mean,
        #     coord3,
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.4,
        #     color=color,
        #     linestyle="-",
        #     marker='.',
        #     linewidth=1,
        #     zorder=10,
        # )
        ax.plot(
            direct_mean,
            direct_coord3,
            color=color,
            alpha=1,
            linestyle="-",
            linewidth=3,
            zorder=10,
        )

        # ax.fill_betweenx(
        #     coord3,
        #     (var_profile_mean - var_profile_std_of_mean),
        #     (var_profile_mean + var_profile_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

        ax.fill_betweenx(
            direct_coord3,
            (direct_mean - direct_std_of_mean),
            (direct_mean + direct_std_of_mean),
            # color=[0.3, 0.3, 0.3],
            alpha=0.1,
            color=color,
            zorder=3,
        )

# ax.set_xticks([60, 80, 100])
# axs[0].set_xlim(0.0, 0.2)
# ax.axvline(100, color="k", linestyle="--", alpha=0.5)
# ax.legend(loc="upper left")


fig.savefig(fig_dir / "mass_fraction_all.svg", transparent=True)

In [ ]:
dataset["max_gridbox"]

In [ ]:
variable = "mass_difference_timestep"
keys = ["null_microphysics", "condensation", "collision_condensation", "coalbure_condensation_large"]
colors = ["black", "red", "blue", "green"]
time_slice = slice(2500, None)

units_conversion = 1 / (20**3) * 1 / 2 * 1e3  # from kg s-1 to g m-3 s-1
monitor_conversion = 1 / (20**3) * 1 / 2 * 1e18  # from g      to g m-3 s-1

latent_heat = 2.25 * 1e6 * 1e-3  # J/g

monitor_conversion = monitor_conversion * latent_heat

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))


for microphysic, color in zip(keys, colors):
    dataset = data_dict[microphysic]["dataset"].sel(time=time_slice)
    ids = dataset["cloud_id"].where((dataset["max_gridbox"] == 33), drop=True)
    dataset = dataset.sel(cloud_id=ids)

    max_gridbox = dataset["max_gridbox"]

    sub_cloud_mask = (dataset["gridbox"] < max_gridbox) & (dataset["gridbox"] > 0)
    coord3 = dataset["pseudo_coord3_full"]

    direct_coord3 = coord3.sel(gridbox=slice(direct["gbxindex"].min(), direct["gbxindex"].max()))

    var_profile = units_conversion * dataset[variable].sum("radius_bins").where(sub_cloud_mask)

    var_profile_mean = var_profile.median("time", skipna=False)
    var_profile_mean = var_profile_mean  # - var_profile_mean.sel(gridbox = max_gridbox - 1)
    var_profile_std_of_mean = var_profile.std("time", skipna=False) / np.sqrt(var_profile["time"].size)

    ax.plot(
        var_profile_mean.mean("cloud_id"),
        coord3.mean("cloud_id"),
        # color=[0.3, 0.3, 0.3],
        alpha=1,
        color=color,
        linestyle="-",
        marker="",
        linewidth=3,
        zorder=10,
        label=data_dict[microphysic]["microphysics"],
    )

    for cloud_id in ids:
        # zarr = xr.open_dataset(f"/home/m/m301096/CLEO/data/output_v3.5/{microphysic}/clusters_{cloud_id}/eurec4a1d_sol.zarr", engine = "zarr",  consolidated=False)
        # zarr = zarr.sel(time = time_slice)
        # direct = zarr["massdelta_cond"].sel(gbxindex = slice(1, dataset["max_gridbox"].sel(cloud_id = cloud_id) -1)) * monitor_conversion
        # direct_mean = direct.mean("time")
        # direct_std_of_mean = (direct.std("time") / np.sqrt(direct["time"].size))

        # axs[0].plot(
        #     var_profile.T,
        #     coord3,
        #     color=color,
        #     alpha=0.01,
        #     linestyle="",
        #     marker='.',
        #     zorder=1,

        # )
        # for ax in axs:
        ax.plot(
            var_profile_mean.sel(cloud_id=cloud_id),
            coord3.sel(cloud_id=cloud_id),
            # color=[0.3, 0.3, 0.3],
            alpha=0.4,
            color=adjust_lightness_array([color], 1.2)[0],
            linestyle="-",
            # marker='.',
            linewidth=1,
            zorder=10,
        )

        # ax.fill_betweenx(
        #     coord3,
        #     (var_profile_mean - var_profile_std_of_mean),
        #     (var_profile_mean + var_profile_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

        # ax.plot(
        #     direct_mean,
        #     direct_coord3,
        #     color=color,
        #     alpha=1,
        #     linestyle="-",
        #     linewidth=3,
        #     zorder=10,
        # )

        # ax.fill_betweenx(
        #     direct_coord3,
        #     (direct_mean - direct_std_of_mean),
        #     (direct_mean + direct_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

# ax.set_xticks([60, 80, 100])
# axs[0].set_xlim(0.0, 0.2)
# ax.axvline(100, color="k", linestyle="--", alpha=0.5)
# ax.legend(loc="upper left")

ax.legend(loc="upper left")
ax.set_xlabel("Evaporation rate in subcloud layer [W m$^{-3}$]")
ax.set_ylabel("Height [m]")
# ax.set_title("Evaporation rate in subcloud layer")

fig.savefig(fig_dir / "watt_evaporation_rate_gbx33.svg", transparent=True)

In [ ]:
variable = "mass_difference_timestep"
keys = ["null_microphysics", "condensation", "collision_condensation", "coalbure_condensation_large"]
colors = ["black", "red", "blue", "green"]
time_slice = slice(2500, None)

units_conversion = 1 / (20**3) * 1 / 2 * 1e3  # from kg s-1 to g m-3 s-1
monitor_conversion = 1 / (20**3) * 1 / 2 * 1e18  # from g      to g m-3 s-1


fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))


for microphysic, color in zip(keys, colors):
    dataset = data_dict[microphysic]["dataset"].sel(time=time_slice)
    ids = dataset["cloud_id"].where((dataset["max_gridbox"] == 33), drop=True)
    dataset = dataset.sel(cloud_id=ids)

    max_gridbox = dataset["max_gridbox"]

    sub_cloud_mask = (dataset["gridbox"] < max_gridbox) & (dataset["gridbox"] > 0)
    coord3 = dataset["pseudo_coord3_full"]

    direct_coord3 = coord3.sel(gridbox=slice(direct["gbxindex"].min(), direct["gbxindex"].max()))

    var_profile = units_conversion * dataset[variable].sum("radius_bins").where(sub_cloud_mask)

    var_profile_mean = var_profile.median("time", skipna=False)
    var_profile_mean = var_profile_mean  # - var_profile_mean.sel(gridbox = max_gridbox - 1)
    var_profile_std_of_mean = var_profile.std("time", skipna=False) / np.sqrt(var_profile["time"].size)

    ax.plot(
        var_profile_mean.mean("cloud_id"),
        coord3.mean("cloud_id"),
        # color=[0.3, 0.3, 0.3],
        alpha=1,
        color=color,
        linestyle="-",
        marker="",
        linewidth=3,
        zorder=10,
        label=data_dict[microphysic]["microphysics"],
    )

    for cloud_id in ids:
        # zarr = xr.open_dataset(f"/home/m/m301096/CLEO/data/output_v3.5/{microphysic}/clusters_{cloud_id}/eurec4a1d_sol.zarr", engine = "zarr",  consolidated=False)
        # zarr = zarr.sel(time = time_slice)
        # direct = zarr["massdelta_cond"].sel(gbxindex = slice(1, dataset["max_gridbox"].sel(cloud_id = cloud_id) -1)) * monitor_conversion
        # direct_mean = direct.mean("time")
        # direct_std_of_mean = (direct.std("time") / np.sqrt(direct["time"].size))

        # axs[0].plot(
        #     var_profile.T,
        #     coord3,
        #     color=color,
        #     alpha=0.01,
        #     linestyle="",
        #     marker='.',
        #     zorder=1,

        # )
        # for ax in axs:
        ax.plot(
            var_profile_mean.sel(cloud_id=cloud_id),
            coord3.sel(cloud_id=cloud_id),
            # color=[0.3, 0.3, 0.3],
            alpha=0.4,
            color=adjust_lightness_array([color], 1.2)[0],
            linestyle="-",
            # marker='.',
            linewidth=1,
            zorder=10,
        )

        # ax.fill_betweenx(
        #     coord3,
        #     (var_profile_mean - var_profile_std_of_mean),
        #     (var_profile_mean + var_profile_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

        # ax.plot(
        #     direct_mean,
        #     direct_coord3,
        #     color=color,
        #     alpha=1,
        #     linestyle="-",
        #     linewidth=3,
        #     zorder=10,
        # )

        # ax.fill_betweenx(
        #     direct_coord3,
        #     (direct_mean - direct_std_of_mean),
        #     (direct_mean + direct_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

# ax.set_xticks([60, 80, 100])
# axs[0].set_xlim(0.0, 0.2)
# ax.axvline(100, color="k", linestyle="--", alpha=0.5)
# ax.legend(loc="upper left")

ax.legend(loc="upper left")
ax.set_xlabel("Evaporation rate in subcloud layer [g m$^{-3}$ s$^{-1}$]")
ax.set_ylabel("Height [m]")
# ax.set_title("Evaporation rate in subcloud layer")

fig.savefig(fig_dir / "evaporation_rate_gbx33.svg", transparent=True)

In [ ]:
variable = "mass_difference_timestep"
keys = ["null_microphysics", "condensation", "collision_condensation", "coalbure_condensation_large"]
colors = ["black", "red", "blue", "green"]
time_slice = slice(1500, None)

units_conversion = 1 / (20**3) * 1 / 2 * 1e3  # from kg s-1 to g m-3 s-1
monitor_conversion = 1 / (20**3) * 1 / 2 * 1e18  # from g      to g m-3 s-1


fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))


for microphysic, color in zip(keys, colors):
    dataset = data_dict[microphysic]["dataset"].sel(time=time_slice)
    ids = dataset["cloud_id"].where((dataset["max_gridbox"] == 33), drop=True)
    dataset = dataset.sel(cloud_id=ids)

    max_gridbox = dataset["max_gridbox"]

    sub_cloud_mask = (dataset["gridbox"] < max_gridbox) & (dataset["gridbox"] > 0)
    coord3 = dataset["pseudo_coord3_full"]

    direct_coord3 = coord3.sel(gridbox=slice(direct["gbxindex"].min(), direct["gbxindex"].max()))

    var_profile = units_conversion * dataset[variable].sum("radius_bins").where(sub_cloud_mask)

    var_profile_mean = var_profile.median("time", skipna=False)
    var_profile_mean = var_profile_mean - var_profile_mean.mean("gridbox")
    var_profile_std_of_mean = var_profile.std("time", skipna=False) / np.sqrt(var_profile["time"].size)

    ax.plot(
        var_profile_mean.mean("cloud_id"),
        coord3.mean("cloud_id"),
        # color=[0.3, 0.3, 0.3],
        alpha=1,
        color=color,
        linestyle="-",
        marker="",
        linewidth=3,
        zorder=10,
        label=data_dict[microphysic]["microphysics"],
    )

    for cloud_id in ids:
        # zarr = xr.open_dataset(f"/home/m/m301096/CLEO/data/output_v3.5/{microphysic}/clusters_{cloud_id}/eurec4a1d_sol.zarr", engine = "zarr",  consolidated=False)
        # zarr = zarr.sel(time = time_slice)
        # direct = zarr["massdelta_cond"].sel(gbxindex = slice(1, dataset["max_gridbox"].sel(cloud_id = cloud_id) -1)) * monitor_conversion
        # direct_mean = direct.mean("time")
        # direct_std_of_mean = (direct.std("time") / np.sqrt(direct["time"].size))

        # axs[0].plot(
        #     var_profile.T,
        #     coord3,
        #     color=color,
        #     alpha=0.01,
        #     linestyle="",
        #     marker='.',
        #     zorder=1,

        # )
        # for ax in axs:
        ax.plot(
            var_profile_mean.sel(cloud_id=cloud_id),
            coord3.sel(cloud_id=cloud_id),
            # color=[0.3, 0.3, 0.3],
            alpha=0.4,
            color=adjust_lightness_array([color], 1.2)[0],
            linestyle="-",
            # marker='.',
            linewidth=1,
            zorder=10,
        )

        # ax.fill_betweenx(
        #     coord3,
        #     (var_profile_mean - var_profile_std_of_mean),
        #     (var_profile_mean + var_profile_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

        # ax.plot(
        #     direct_mean,
        #     direct_coord3,
        #     color=color,
        #     alpha=1,
        #     linestyle="-",
        #     linewidth=3,
        #     zorder=10,
        # )

        # ax.fill_betweenx(
        #     direct_coord3,
        #     (direct_mean - direct_std_of_mean),
        #     (direct_mean + direct_std_of_mean),
        #     # color=[0.3, 0.3, 0.3],
        #     alpha=0.1,
        #     color=color,
        #     zorder=3,
        # )

# ax.set_xticks([60, 80, 100])
# axs[0].set_xlim(0.0, 0.2)
# ax.axvline(100, color="k", linestyle="--", alpha=0.5)
# ax.legend(loc="upper left")

ax.legend(loc="upper left")
ax.set_xlabel("Evaporation rate deviation in subcloud layer [g m$^{-3}$ s$^{-1}$]")
ax.set_ylabel("Height [m]")
ax.set_title("Evaporation rate deviation in subcloud layer")

fig.savefig(fig_dir / "fraction_evaporation_rate_gbx33.svg", transparent=True)

In [ ]:
variable = "mass_difference"
keys = ["collision_condensation", "coalbure_condensation_large", "null_microphysics", "condensation"]
colors = ["black", "red", "blue", "green"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

for key, color in zip(keys, colors):
    dataset = data_dict[key]["dataset"].sel(time=slice(2500, None))

    for cloud_id in [222, 142, 301]:
        sub_cloud_mask = (dataset["gridbox"] < dataset["max_gridbox"].sel(cloud_id=cloud_id)) & (
            dataset["gridbox"] > 0
        )
        coord3 = dataset["pseudo_coord3_full"].sel(cloud_id=cloud_id)
        var_profile = dataset[variable].sel(cloud_id=cloud_id).sum("radius_bins").where(sub_cloud_mask)

        var_profile_mean = var_profile.median("time", skipna=False)
        var_profile_std_of_mean = var_profile.std("time", skipna=False) / np.sqrt(
            var_profile["time"].size
        )

        # axs[0].plot(
        #     var_profile.T,
        #     coord3,
        #     color=color,
        #     alpha=0.01,
        #     linestyle="",
        #     marker='.',
        #     zorder=1,

        # )
        # for ax in axs:
        ax.plot(
            var_profile_mean,
            coord3,
            # color=[0.3, 0.3, 0.3],
            alpha=0.4,
            color=color,
            linestyle="-",
            marker=".",
            linewidth=1,
            zorder=10,
        )
        ax.fill_betweenx(
            coord3,
            (var_profile_mean - var_profile_std_of_mean),
            (var_profile_mean + var_profile_std_of_mean),
            # color=[0.3, 0.3, 0.3],
            alpha=0.1,
            color=color,
            zorder=3,
        )

# ax.set_xticks([60, 80, 100])
# axs[0].set_xlim(0.0, 0.2)
# ax.axvline(100, color="k", linestyle="--", alpha=0.5)
# ax.legend(loc="upper left")


fig.savefig(fig_dir / "mass_fraction_all.svg", transparent=True)

### Vertical profiles of LWC fraction

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["no_physics_1024", "condensation_1024", "collision_condensation_1024"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")
ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_1024.svg", transparent=True)

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["condensation_512", "condensation_1024", "condensation_2048"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_condensation.svg", transparent=True)

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["condensation_1024"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_condensation_1024.svg", transparent=True)

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["condensation_2048"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_condensation_2048.svg", transparent=True)

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["condensation_1024"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics_all(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_condensation_1024_all.svg", transparent=True)

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["collision_condensation_512", "collision_condensation_1024", "collision_condensation_2048"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")
ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_collision.svg", transparent=True)

In [ ]:
variable = "vertical_profile_lwc_fraction"
keys = ["no_physics_512", "no_physics_1024"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

plot_single_microphysics(
    ax=ax,
    data_dict=data_dict,
    data_keys=keys,
    variable=variable,
    cloud_dict=clouds_dict,
)

ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

ax.set_xlabel(r"Evaporated mass $g m^{-3}$")

fig.savefig(fig_dir / "evaporated_mass_null_microphysics.svg", transparent=True)

### Number of superdroplets

In [ ]:
variable = "number_superdroplets"
keys = ["collision_condensation_512", "collision_condensation_1024", "collision_condensation_2048"]

fig, axs = plt.subplots(ncols=len(keys), nrows=1, figsize=(15, 4.5))

idx = 0
for key in keys:
    dataset = data_dict[key]["dataset"]
    linestyle = "-"
    color = data_dict[key]["color"]
    ax = axs[idx]

    dataset = data_dict[key]["dataset"]
    dataset = dataset.sel(cloud_id=dataset["max_gridbox"] < 40)
    dataset = dataset.sel(gridbox=slice(None, 40))
    dataset = dataset.sel(time=slice(400, None))
    data = dataset[variable]
    data = data.where(data <= 1e6)
    data = data.sum("radius_bins").mean("time")
    data = data.where(data != 0)
    ax.plot(
        data.T,
        dataset["pseudo_coord3"],
        label=key,
        linestyle=linestyle,
        color=color,
        alpha=0.1,
    )
    ax.plot(
        data.quantile(0.5, "cloud_id"),
        dataset["pseudo_coord3"],
        label=key,
        linestyle=linestyle,
        alpha=1,
        color=color,
    )

    title = data_dict[key]["microphysics"]
    ax.set_title(title)

    idx += 1

for ax in axs:
    ax.set_xlim(0, 400)
    ax.set_xlabel("Number of Superdroplets")
    ax.set_ylabel("Height $m$")

fig.tight_layout()
fig.savefig(fig_dir / "number_superdroplets_collision.svg", transparent=True)

/work/mh1126/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/mh1126/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/mh1126/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [ ]:
variable = "number_superdroplets"
keys = ["collision_condensation_512", "collision_condensation_1024", "collision_condensation_2048"]

fig, axs = plt.subplots(ncols=len(keys), nrows=1, figsize=(15, 4.5))

idx = 0
for key in keys:
    dataset = data_dict[key]["dataset"]
    linestyle = "-"
    color = data_dict[key]["color"]
    ax = axs[idx]

    dataset = data_dict[key]["dataset"]
    dataset = dataset.sel(cloud_id=dataset["max_gridbox"] < 40)
    dataset = dataset.sel(gridbox=slice(None, 40))
    dataset = dataset.sel(time=slice(400, None))
    data = dataset[variable]
    data = data.where(dataset["sub_cloud_layer_mask"])
    data = data.where(data <= 1e6)
    data = data.sum("radius_bins").mean("time")
    data = data.where(data != 0)

    ax.plot(
        data.T,
        dataset["pseudo_coord3"],
        label=key,
        linestyle=linestyle,
        color=color,
        alpha=0.1,
    )
    ax.plot(
        data.quantile(0.5, "cloud_id"),
        dataset["pseudo_coord3"],
        label=key,
        linestyle=linestyle,
        alpha=1,
        color=color,
    )

    title = data_dict[key]["microphysics"]
    ax.set_title(title)

    idx += 1

for ax in axs:
    ax.set_xlim(0, 400)
    ax.set_xlabel("Number of Superdroplets")
    ax.set_ylabel("gridbox")

fig.tight_layout()
fig.savefig(fig_dir / "number_superdroplets_collision_no_cloud_layer.svg", transparent=True)

/work/mh1126/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/mh1126/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/mh1126/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


### Plots to clarify issues with calculations

#### Mass represented

In [ ]:
key = "null_microphysics"
variable = "mass_represented"

dataset = data_dict[key]["dataset"]
for cloud_id in clouds_dict:
    ds = dataset.sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))

gridboxes = ds["gridbox"].isel(gridbox=[-1, -2, -3, -5, 10, 0])
altitudes = ds["pseudo_coord3"].sel(gridbox=gridboxes)
fig, axs = plt.subplots(ncols=len(gridboxes) + 1, nrows=1, figsize=(15, 10), sharex=True, sharey=True)

my_cmap = mpl.colormaps["plasma"].resampled(20)
my_norm = mpl.colors.LogNorm(vmin=1e-2, vmax=1e0)

for idx, gridbox in enumerate(gridboxes):
    altitude = ds["pseudo_coord3"].sel(gridbox=gridbox)
    ds_selected = ds.sel(gridbox=gridbox)
    pcm = axs[idx].pcolormesh(
        ds_selected[variable]["radius_bins"],
        ds_selected[variable]["time"],
        ds_selected[variable],
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )
    axs[idx].set_title(f"Gridbox {gridbox.data}\n{altitude.data}m")
for ax in axs:
    ax.axvline(50)
    ax.set_xticks([1e0, 1e1, 1e2, 1e3])
    ax.set_xlim(1e0, 1e3)
    ax.grid()
    ax.set_xlabel("Radius $\\mu m$")
axs[0].set_xscale("log")

axs[0].set_ylabel("Time $s$")

fig.colorbar(
    ax=axs[-1], mappable=pcm, orientation="vertical", label=f"{variable} {ds[variable].attrs['units']}"
)

fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

Text(0.5, 0.98, 'Cloud 142 - Null microphysics (2048 SDs) - mass_represented')

#### Functions

In [ ]:
def temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))
    fig, axs = plt.subplots(
        ncols=2, nrows=1, figsize=(15, 7), sharex=False, sharey=True, width_ratios=[1, 0.5]
    )

    pcm = axs[0].pcolormesh(
        ds[variable]["time"],
        ds["pseudo_coord3_full"],
        ds[variable].sum("radius_bins").T,
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )
    fig.colorbar(
        mappable=pcm, ax=axs[0], label=f"{variable} {ds[variable].attrs['units']}", extend="both"
    )
    axs[0].set_xlabel("Time $s$")
    axs[0].set_ylabel("Height $m$")

    ds_stationary = ds.sel(time=slice(400, None))

    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").quantile(0.5, "time"),
        ds_stationary["pseudo_coord3_full"],
    )
    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").T,
        ds_stationary["pseudo_coord3_full"],
        color=[0.7, 0.7, 0.7],
        linewidth=0.1,
        alpha=0.1,
        zorder=0,
    )
    axs[1].fill_betweenx(
        ds_stationary["pseudo_coord3_full"],
        ds_stationary[variable].sum("radius_bins").quantile(0.25, "time"),
        ds_stationary[variable].sum("radius_bins").quantile(0.75, "time"),
        color="k",
        alpha=0.1,
    )
    axs[1].set_xlim(vmin, vmax)
    axs[1].set_xlabel(f"{variable} {ds[variable].attrs['units']}")
    axs[1].set_ylabel("Height $m$")

    fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

    return fig, axs


def temporal_evolution_of_variable_dataarray(ds, variable, cloud_id, vmin, vmax):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    fig, axs = plt.subplots(
        ncols=2, nrows=1, figsize=(15, 7), sharex=False, sharey=True, width_ratios=[1, 0.5]
    )

    pcm = axs[0].pcolormesh(
        ds["time"],
        ds["pseudo_coord3_full"],
        ds[variable].T,
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )
    fig.colorbar(
        mappable=pcm, ax=axs[0], label=f"{variable} {ds[variable].attrs['units']}", extend="both"
    )
    axs[0].set_xlabel("Time $s$")
    axs[0].set_ylabel("Height $m$")

    ds_stationary = ds.sel(time=slice(1500, None))

    axs[1].plot(
        ds_stationary[variable].quantile(0.5, "time"),
        ds_stationary["pseudo_coord3_full"],
    )
    axs[1].plot(
        ds_stationary[variable].T,
        ds_stationary["pseudo_coord3_full"],
        color=[0.7, 0.7, 0.7],
        linewidth=0.1,
        alpha=0.1,
        zorder=0,
    )
    axs[1].fill_betweenx(
        ds_stationary["pseudo_coord3_full"],
        ds_stationary[variable].quantile(0.25, "time"),
        ds_stationary[variable].quantile(0.75, "time"),
        color="k",
        alpha=0.1,
    )
    axs[1].set_xlim(vmin, vmax)
    axs[1].set_xlabel(f"{variable} {ds[variable].attrs['units']}")
    axs[1].set_ylabel("Height $m$")

    fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

    return fig, axs


def temporal_evolution_different_gridboxes(
    data_dict, key, variable, cloud_id, vmin, vmax, isel_gridboxes=[-1, -2, -3, -5, 10, 0]
):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))

    gridboxes = ds["gridbox"].isel(gridbox=isel_gridboxes)
    fig, axs = plt.subplots(
        ncols=len(gridboxes) + 1, nrows=1, figsize=(15, 10), sharex=True, sharey=True
    )

    for idx, gridbox in enumerate(gridboxes):
        altitude = ds["pseudo_coord3_full"].sel(gridbox=gridbox)
        ds_selected = ds.sel(gridbox=gridbox)
        pcm = axs[idx].pcolormesh(
            ds_selected[variable]["radius_bins"],
            ds_selected[variable]["time"],
            ds_selected[variable],
            shading="auto",
            cmap=my_cmap,
            norm=my_norm,
        )
        axs[idx].set_title(f"Gridbox {gridbox.data}\n{altitude.data}m")
    for ax in axs:
        ax.set_xticks([1e0, 1e1, 1e2, 1e3])
        ax.set_xlim(1e0, 1e3)
        ax.grid()
        ax.set_xlabel("Radius $\\mu m$")
    axs[0].set_xscale("log")

    axs[0].set_ylabel("Time $s$")

    fig.colorbar(
        ax=axs[-1],
        mappable=pcm,
        orientation="vertical",
        label=f"{variable} {ds[variable].attrs['units']}",
        extend="both",
    )
    fig.suptitle(f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {variable}")

    return fig, axs

#### Mass represented

In [ ]:
vmin, vmax = 1e-10, 1.5
variable = "mass_represented"
cloud_id = 222
key = "null_microphysics"

fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax / 5)

In [ ]:
vmin, vmax = 1e-10, 1.5
variable = "mass_represented"
cloud_id = 222
key = "null_microphysics"

fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax / 5)

#### Multiplicity

In [ ]:
vmin, vmax = 1e-10, 5e6
variable = "xi"
cloud_id = 222
key = "null_microphysics"

fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax / 5)

#### Number superdroplets

In [ ]:
vmin, vmax = 1e-10, 240
variable = "number_superdroplets"
cloud_id = 222
key = "null_microphysics"

fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax / 10)